In [1]:
'''

整體功能描述

'''

'\n\n整體功能描述\n\n'

In [2]:
'''

Application 主架構

'''

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError)

# 載入系統所需處理套件
import json
import os
import time

# 載入模型套件
import pickle

# 載入文字處理套件
from jieba import cut

# 載入圖形處理套件
from linebot.models import ImageMessage
import glob
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions


# 載入基礎設定檔
secretFileContentJson=json.load(open("./line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/素材" , static_folder = "./素材/")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

Using TensorFlow backend.


In [3]:
'''

消息判斷器

讀取指定的json檔案後，把json解析成不同格式的SendMessage

讀取檔案，
把內容轉換成json
將json轉換成消息
放回array中，並把array傳出。

'''

# 引用會用到的套件
from linebot.models import (
    ImagemapSendMessage,TextSendMessage,ImageSendMessage,LocationSendMessage,FlexSendMessage
)

from linebot.models.template import (
    ButtonsTemplate,CarouselTemplate,ConfirmTemplate,ImageCarouselTemplate
    
)

from linebot.models.template import *

def detect_json_array_to_new_message_array(fileName):
    
    #開啟檔案，轉成json
    with open(fileName) as f:
        jsonArray = json.load(f)
    
    # 解析json
    returnArray = []
    for jsonObject in jsonArray:

        # 讀取其用來判斷的元件
        message_type = jsonObject.get('type')
        
        # 轉換
        if message_type == 'text':
            returnArray.append(TextSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'imagemap':
            returnArray.append(ImagemapSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'template':
            returnArray.append(TemplateSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'image':
            returnArray.append(ImageSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'sticker':
            returnArray.append(StickerSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'audio':
            returnArray.append(AudioSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'location':
            returnArray.append(LocationSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'flex':
            returnArray.append(FlexSendMessage.new_from_json_dict(jsonObject))    

    # 回傳
    return returnArray

def detect_json_array_to_new_message_array_2(jsonArray):

    # 解析json
    returnArray = []
    for jsonObject in jsonArray:

        # 讀取其用來判斷的元件
        message_type = jsonObject.get('type')
        
        # 轉換
        if message_type == 'text':
            returnArray.append(TextSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'imagemap':
            returnArray.append(ImagemapSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'template':
            returnArray.append(TemplateSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'image':
            returnArray.append(ImageSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'sticker':
            returnArray.append(StickerSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'audio':
            returnArray.append(AudioSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'location':
            returnArray.append(LocationSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'flex':
            returnArray.append(FlexSendMessage.new_from_json_dict(jsonObject))    

    # 回傳
    return returnArray

In [4]:
'''
定義隨機文章推播功能

'''
import pandas as pd
import random

def findnews(label, labellist):
    news_df = pd.read_csv('./news/ieknews-data.csv', encoding='utf-8')
    news_df.set_index('seq', inplace=True)
    newsid = random.sample(labellist, 5)
    newsid = pd.Series(newsid)
    news_df = news_df.loc[newsid, ['otitle', 'odate', 'url']]
    contentsList = []
    num = 0
    try:        
        # 隨機選單
        
        bubblerandom = {
              #"type": "flex",
              #"altText": "Flex Message",
              #"contents": {
                "type": "bubble",
                "hero": {
                  "type": "image",
                  "url": "https://et-storage-1.s3-ap-northeast-1.amazonaws.com/ICON/random.jpg",
                  "size": "xl",
                  "aspectRatio": "1:1",
                  "aspectMode": "cover",
                  "action": {
                    "type": "postback",
                    "label": "隨機搜尋",
                    "text": "隨機搜尋",
                    "data": "news=" + str(label)
                  }
                },
                "body": {
                  "type": "box",
                  "layout": "vertical",
                  "spacing": "md",
                  "contents": [
                    {
                      "type": "text",
                      "text": "隨機搜尋",
                      "size": "xl",
                      "align": "center",
                      "gravity": "center",
                      "weight": "bold",
                      "color": "#524D4D",
                      "action": {
                        "type": "postback",
                        "label": "隨機搜尋",
                        "text": "隨機搜尋",
                        "data": "news=" + str(label)
                      }#,
                      #"wrap": true
                    },
                    {
                      "type": "text",
                      "text": "重新呈現另外五篇文章",
                      "size": "sm",
                      "align": "center"
                    }
                  ]
                }
              }
            #}
           
        
        # 將資料製作成隨機選單json
        for index, i in news_df.iterrows():
            num += 1
            print(i['otitle'], i['odate'], i['url'])
            
           
            # 循環加到 list_col_color
            bubbleDict = {
                    "type": "bubble",
                    "hero": {
                      "type": "image",
                      "url": "https://et-storage-1.s3-ap-northeast-1.amazonaws.com/ICON/news" + str(num) + ".JPG",
                      "size": "xl",
                      "aspectRatio": "1:1",
                      "aspectMode": "cover",
                      "action": {
                        "type": "uri",
                        "uri": i['url']  #文章url
                      }
                    },
                    "body": {
                      "type": "box",
                      "layout": "vertical",
                      "spacing": "md",
                      "contents": [
                        {
                          "type": "text",
                          "text": i['otitle'],  #文章title
                          "size": "md",
                          "align": "center",
                          "gravity": "center",
                          "weight": "bold",
                          "color": "#524D4D",
                          "action": {
                            "type": "uri",
                            "uri": i['url']  #文章url
                          }#,
                          #"wrap": true
                        },
                        {
                          "type": "text",
                          "text": i['odate'].split(' ')[0],  #文章時間
                          "align": "center",
                          "weight": "bold"
                        },
                        {
                          "type": "box",
                          "layout": "vertical",
                          "contents": [
                            {
                              "type": "button",
                              "size": "md",
                              "action": {
                                "type": "uri",
                                "label": "開啟",
                                "uri": i['url']  #文章url
                              },
                              "style": "primary"
                            },
                            {
                              "type": "button",
                              "size": "md",
                              "action": {
                                "type": "postback",
                                "label": "我不喜歡",
                                "data": "default"
                              }
                            }
                          ]
                        }
                      ]
                    }
                  }
                  
            contentsList.append(bubbleDict) #插入隨選文章選項 
        
        contentsList.append(bubblerandom)  #插入random選項
        
        Flex_Message = [
            {
              "type": "flex",
              "altText": "Flex Message",
              "contents": {
                "type": "carousel",
                "contents":contentsList
              }
            }
        ]
           
        result_message_array = detect_json_array_to_new_message_array_2(Flex_Message)
    
    # 回傳資訊
    finally:
        return result_message_array

In [5]:
"""

定義薪資預測的爬蟲功能
傳入職缺網址，ETL 薪資預測模型所需資料欄位，回傳DataFrame

return :
    來源,職務大類,管理責任,出差外派,工作時間,最高薪資,最低薪資,工作經歷,最低學歷,應徵人數,工作地點區域,產業類型大類(company),產業類型中類(company)
    source,category_top,management,expatriate,work_time,salary_max,salary_min,accept_experience,education_min,candidates,area,industrial_top,industrial_mid
    ex: 104,管理財經,No,No,日班,32000.0,28000.0,3,高中,0~10人應徵,雲嘉南,文教傳播,教育服務

"""

import re
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import lxml
import pandas as pd
import csv
import json
import time
import ssl
import pickle
ssl._create_default_https_context = ssl._create_unverified_context


### 爬取104職缺的網頁資訊(針對一個url)|output:dictionary ###
def crawlerJob104(url):
    # 找到內容網址
    head = {'User-Agent': 'Mozilla/5.0'}
    res = requests.get(url, headers=head)
    time.sleep(0.3)  # 休息供網頁讀取
    html = BeautifulSoup(res.text, 'lxml')
    saved = {}

    # 資料區塊
    sec = html.find_all("section", class_="info")  # 主要區塊list
    if not sec :
        saved["Oops"] = "已經結束徵才"
        return {}

    s1 = sec[0]  # 工作內容區塊
    s2 = sec[1]  # 條件要求區塊

    # 開始抓資料囉：公司名稱、應徵人數
    compl = html.find("a", class_="cn")["href"]
    vol = html.find("div", class_="sub").find_all("a")[0].text.replace("\n", "")
    saved["公司連結"] = compl
    saved["應徵人數"] = vol

    # 開始抓資料囉：工作內容
    workcont = {}
    wclc = s1.find_all("dt")  # [欄位]list
    wcl = s1.find_all("dd")  # [資料]list

    wc = s1.find("div", class_="content").find("p")
    wc = wc.prettify().replace("<p>", "").replace("</p>", "").replace("<br/>", "").replace("\n", "").strip()
    workcont["工作內容"] = wc

    for i in range(len(wclc)):
        if "職務類別" in wclc[i].text:
            jc = wcl[i].find_all("span", class_="")
            jc = str(jc).replace("<span>", "").replace("</span>", "").replace("[", "").replace("]", "")
            workcont["職務類別"] = jc
        elif "工作待遇" in wclc[i].text:
            sa = wcl[i].text.split("\n")[0].strip()
            workcont["工作待遇"] = sa
        elif "上班地點" in wclc[i].text:
            lc = wcl[i].prettify().split("\n")[1].strip()
            workcont["上班地點"] = lc
        else:
            culumn = wclc[i].text.replace("：", "")
            workcont[culumn] = wcl[i].text.replace("\n", "").strip()

    saved["工作內容"] = workcont

    # 開始抓資料囉：條件要求
    workreq = {}
    wqc = s2.find_all("dt")  # [欄位]list
    wql = s2.find_all("dd")  # [資料]list

    # 條件列表與內容(每個職缺不同，故寫迴圈)
    for i in range(len(wql)):
        culumn = wqc[i].text.replace("：", "")
        workreq[culumn] = wql[i].text.replace("\n", "").replace("\r", "").strip()

    saved["條件要求"] = workreq

    return saved


### 爬取1111職缺的網頁資訊(針對一個url)|output:dictionary ###
def crawlerJob1111(url):
    jobno = url.split('/')[-2]
    res = requests.get(url)
    html = BeautifulSoup(res.text, 'lxml')

    # data dict
    saved = {}

    # 職稱
    logo = html.find("div", class_="logoTitle")
    if logo == None:
        saved["Oops"] = "已經結束徵才"
        return {}

    #saved['title'] = logo.find("h1").text
    # 公司link
    companylink = logo.find("li", class_="ellipsis").find("a")["href"]
    saved['公司連結'] = "https://www.1111.com.tw" + companylink

    incontent = html.find("section", id="incontent")

    # 工作內容
    cont1 = {}
    workcont = incontent.find("ul", class_="dataList")

    wcontLis = workcont.find_all("li")
    for li in wcontLis:
        try:
            lli = li.find_all("div")
            lliname = lli[0].text.replace('：', '')
            ds = lli[1].find_all("div", class_="cateshow")
            for d in ds:
                d.extract()
            ds = lli[1].find_all("a")
            for d in ds:
                d.extract()
            cont1[lliname] = (lli[1].text.replace(' ', '')).replace('　', '')
        except:
            pass
    saved['工作內容'] = cont1

    # 工作條件
    cont3 = {}
    condi = incontent.find("article", class_="boxsize")
    wcondiLis = condi.find_all("li")
    for li in wcondiLis:
        try:
            lli = li.find_all("div")
            lliname = lli[0].text.replace('：', '')

            cont3[lliname] = lli[1].text.strip()
        except:
            pass
    saved['要求條件'] = cont3

    # 應徵人數
    eurl = "https://www.1111.com.tw/includesU/employeeShowRecruits.asp"
    datas = {'eNo': jobno}
    eres = requests.post(eurl, data=datas)
    eres.encoding = 'utf-8'
    r = BeautifulSoup(eres.text, 'lxml')
    saved['應徵人數'] = r.find_all("span")[-1].text

    return saved


### 爬取104公司的資訊(針對一個url)|output:dictionary ###
def crawlerCompany104(url):
    saved = {}

    # 找到內容網址
    json_url = "https://www.104.com.tw/company/ajax/content/" + url.split("/")[4].split("?")[0]
    res = urlopen(json_url)
    time.sleep(0.3)
    data = json.load(res)

    # 找到資料位置
    saved["產業類型"] = data["data"]["industryDesc"]

    return saved


### 爬取1111公司的資訊(針對一個url)|output:dictionary ###
def crawlerCompany1111(url):
    cont = {}
    try:
        res = requests.get(url)
        html = BeautifulSoup(res.text, 'lxml')
        dataList = html.find("ul", class_="dataList")
        base = dataList.find_all("li")
        for li in base:
            try:
                lli = li.find_all("div")
                lliname = lli[0].text.replace('：', '')
                ds = lli[1].find_all("a", id="showmap")
                for d in ds:
                    d.extract()
                cont[lliname] = (lli[1].text.replace(' ', '')).replace('　', '')
            except:
                pass
    except:
        pass

    return cont


# category_top dict
def getCategoryDict(source):
    if source == '1111':
        csvfile = './data/category_job1111.csv'
    else:
        csvfile = './data/category_job104.csv'
    cg_dict = {}
    with open(csvfile, encoding='utf8') as f:
        rows = csv.reader(f)
        for row in rows:
            cg_dict[row[0]] = row[1]
    return cg_dict


def get_area(location):
    # 建立area字典，取代原area資料，字典沒有的則統一為海外
    area_dict = {'台北': '北北基', '新北': '北北基', '基隆': '北北基',
                 '桃園': '桃竹苗', '新竹': '桃竹苗', '苗栗': '桃竹苗',
                 '台中': '中彰投', '彰化': '中彰投', '南投': '中彰投',
                 '雲林': '雲嘉南', '嘉義': '雲嘉南', '台南': '雲嘉南',
                 '高雄': '高高屏', '屏東': '高高屏',
                 '宜蘭': '宜花東', '花蓮': '宜花東', '台東': '宜花東'}
    area = area_dict.get(location[:2])
    if not area:
        area = '海外'
    return pd.Series([area])


def get_worktime(source, doc):
    result = ''
    doc = doc.replace('上班', '')
    doc = doc.replace('下班', '')
    doc = doc.replace('加班', '')
    doc = doc.replace('常日班', '')
    doc = doc.replace('白天班', '')
    doc = "".join(doc.split())
    cond1 = doc.count('班') > 1 or '輪班' in doc
    cond2 = (doc.count('班') > 1 and '日班日班' not in doc and '不需輪班' not in doc) \
            or True in [x in doc for x in ['輪班', '假日班', '短期']]

    if (source == '104' and cond1) or (source == '1111' and cond2):
        result = '輪班'
    else:
        if '日班' in doc:
            result = '日班'
        elif '大夜班' in doc:
            result = '大夜班'
        elif '晚班' in doc or '中班' in doc or '中班' in doc:
            result = '晚班'
        elif 'AM' in doc:
            result = '日班'
        elif 'PM' in doc:
            result = '晚班'
        elif '~' in doc:
            whour = int(doc.split('~')[0].split(':')[0])
            if whour >= 5 and whour <= 12:
                result = '日班'
            elif whour >= 13 and whour <= 19:
                result = '晚班'
            else:
                result = '大夜班'
        else:
            result = '輪班'

    return pd.Series([result])


def get_categoryTop(source, str):
    result = ''
    cgmap = getCategoryDict(source)
    for doc in str.split('、'):
        for d in doc.split(', '):
            d = d.strip()
            d = d.replace('教育訓綀人員','教育訓練人員')
            d = d.replace('秘書','祕書')
            d = d.replace('/', '／')
            if cgmap.get(d):
                match = True
                result = cgmap.get(d)
                break
        # 取得第一個符合就離開
        if match:
            break
    return pd.Series([result])


def get_manage(manage):
    result = "No"
    if manage:
        if ("不需" in manage) or ("不須" in manage):
            result = "No"
        else:
            result = "Yes"
    return pd.Series([result])


def get_salary_104(doc):
    result = [0, 0] # [min,max]
    if '月薪' in doc:
        salary = doc.replace(',', '').split('~')
        if len(salary) > 1:
            result[0] = salary[0].split(' ')[1]
            result[1] = salary[1].split('元')[0]
        else:
            min = salary[0].split(' ')[1]
            min = min.replace('元', '').replace('以上', '')
            result[0] = min
            result[1] = min
    elif '年薪' in doc:
        salary = doc.replace(',', '').split('~')
        if len(salary) > 1:
            result[0] = int(salary[0].split(' ')[1]) // 12
            result[1] = int(salary[1].split('元')[0]) // 12
        else:
            min = salary[0].split(' ')[1]
            min = min.replace('元', '').replace('以上', '')
            min = int(min) // 12
            result[0] = min
            result[1] = min
    return result


def get_salary_1111(doc):
    result = [0, 0] # [min,max]
    if '月薪' in doc and doc != '月薪':
        salary = doc.replace(',', '').split('元至')
        if len(salary) > 1:
            result[0] = salary[0].split('月薪')[1].replace(')', '')
            result[1] = salary[1].split('元')[0]
        else:
            min = salary[0].split('月薪')[1].replace(')', '').replace('元', '').replace('以上', '')
            result[0] = min
            result[1] = min
    elif '年薪' in doc and doc != '年薪':
        salary = doc.replace(',', '').split('元至')
        if len(salary) > 1:
            result[0] = int(salary[0].split('年薪')[1].replace(')', '')) // 12
            result[1] = int(salary[1].split('元')[0]) // 12
        else:
            min = salary[0].split('年薪')[1].replace(')', '').replace('元', '').replace('以上', '')
            result[0] = int(min) // 12
            result[1] = int(min) // 12
    return result


def get_experi_1111(doc):
    result = 0
    if doc:
        yy = doc.split('年')
        if yy[0].isdigit():
            result = int(yy[0])
        elif '限定' in yy[0]:
            yy1 = yy[0].split('限定 ')[1]
            if yy1.isdigit():
                result = int(yy1)

    return pd.Series([result])


def get_educationMin_104(doc):
    # 最低: 國中
    min = '國中'
    if not doc:
        return pd.Series([min])

    iden = doc.split('、')
    iLen = len(iden)
    if iLen > 1:
        if '高中以下' in iden[0]:
            min = '國中'
        else:
            min = iden[0]
    else:
        if '以下' in iden[0]:
            min = '國中'
        elif '以上' in iden[0]:
            min = iden[0].replace('以上', '')
        else:
            min = iden[0]
            
    if min in ['碩士', '博士']:
        min = '碩士以上'

    return pd.Series([min])


def get_educationMin_1111(doc):
    # 最低: 國中
    min = '國中'
    if not doc:
        return pd.Series([min])

    dd = doc.split(' ')[0].replace('高中職', '高中').replace('國小/國中', '國中').replace('大學、碩士', '大學')
    dd = dd.split('以上')[0]
    if '以上' in doc:
        min = dd
    elif '以下' in doc:
        min = '國中'
    else:
        min = dd
        
    if min in ['碩士', '博士']:
        min = '碩士以上'
    
    return pd.Series([min])


def get_intern_104(doc):
    result = '0~10人應徵'
    if not doc:
        return pd.Series([result])
    if '0~5' in doc or '6~10' in doc:
        result = '0~10人應徵'
    elif '11~30' in doc:
        result = '11~30人應徵'
    elif '30人以上' in doc:
        result = '31人以上應徵'
    return pd.Series([result])


def get_intern_1111(doc):
    result = '0~10人應徵'
    if not doc:
        return pd.Series([result])
    if '0 ~ 10' in doc:
        result = '0~10人應徵'
    elif '11 ~ 30' in doc:
        result = '11~30人應徵'
    elif '31 ~ 50' in doc or '超過 50' in doc:
        result = '31人以上應徵'
    return pd.Series([result])


def etl_104(df, job_dict):
    df['source'] = pd.Series("104")
    # 工作內容
    workcontent = job_dict.get('工作內容')
    if workcontent:
        if workcontent.get('職務類別'):
            df['category_top'] = get_categoryTop('104', workcontent.get('職務類別'))
        df['management'] = get_manage(workcontent.get('管理責任'))
        df['expatriate'] = pd.Series("No")
        if workcontent.get("出差外派"):
            if "無需" not in workcontent.get("出差外派"):
                df['expatriate'] = pd.Series("Yes")
        if workcontent.get('上班時段'):
            df['work_time'] = get_worktime('104', workcontent.get('上班時段'))
        if workcontent.get('工作待遇'):
            salary = get_salary_104(workcontent.get('工作待遇'))
            df['salary_min'] = pd.Series(salary[0])
            df['salary_max'] = pd.Series(salary[1])
        if workcontent.get('上班地點'):
            df['area'] = get_area(workcontent.get('上班地點'))
    # 要求條件
    condition = job_dict.get('條件要求')
    if condition:
        df['education_min'] = get_educationMin_104(condition.get('學歷要求'))
        df['accept_experience'] = 0
        if condition.get('工作經歷'):
            exyear = condition.get('工作經歷').replace('年以上', '')
            if exyear.isdigit():
                df['accept_experience'] = int(exyear)
    # 應徵人數
    df['candidates'] = get_intern_104(job_dict.get('應徵人數'))

    return df


def etl_1111(df, job_dict):
    df['source'] = pd.Series("1111")
    # 工作內容
    workcontent = job_dict.get('工作內容')
    if workcontent:
        if workcontent.get('職務類別'):
            df['category_top'] = get_categoryTop('1111', workcontent.get('職務類別'))
        df['management'] = get_manage(workcontent.get('管理人數'))
        df['expatriate'] = pd.Series("No")
        if workcontent.get("是否出差"):
            df['expatriate'] = pd.Series("Yes")
        if workcontent.get('工作時間'):
            df['work_time'] = get_worktime('1111', workcontent.get('工作時間'))
        if workcontent.get('工作待遇'):
            salary = get_salary_1111(workcontent.get('工作待遇'))
            df['salary_min'] = pd.Series(salary[0])
            df['salary_max'] = pd.Series(salary[1])
        if workcontent.get('工作地點'):
            df['area'] = get_area(workcontent.get('工作地點'))
    # 要求條件
    condition = job_dict.get('要求條件')
    if condition:
        df['education_min'] = get_educationMin_1111(condition.get('學歷限制'))
        if condition.get('工作經驗'):
            df['accept_experience'] = get_experi_1111(condition.get('工作經驗'))
    # 應徵人數
    df['candidates'] = get_intern_1111(job_dict.get('應徵人數'))

    return df


### 取得公司產業類別 ###
def get_industry(source, dust):
    # [top, mid]
    result = ["暫不提供", "暫不提供"]
    if not dust:
        return result

    if source == '1111':
        csvfile = './data/category_industry1111.csv'
    else:
        csvfile = './data/category_industry104.csv'

    cgmap = {}
    with open(csvfile, encoding='utf8') as f:
        rows = csv.reader(f)
        for row in rows:
            cgmap[row[0]] = {
                'L': row[1],
                'M': row[2]
            }
    if source == '104':
        dust = dust.replace('╱', '／').replace('/', '／')
    else:
        dust = dust.split("、")[0]
        dust = dust.replace('╱', '／').replace('/', '／')

    if cgmap.get(dust):
        result[0] = cgmap.get(dust).get('L')
        result[1] = cgmap.get(dust).get('M')

    return result

In [6]:
'''

handler處理關注消息

用戶關注時，讀取 素材 -> 關注 -> reply.json

將其轉換成可寄發的消息，傳回給Line

'''

# 引用套件
from linebot.models import (
    FollowEvent
)

# 關注事件處理
@handler.add(FollowEvent)
def process_follow_event(event):
    
    # 詢問用戶目前身分
    replyJsonPath = "素材/Q目前身分/reply.json"
    result_message_array = []
    result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
        
    # 取出消息內用戶的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
    # 呼叫個人用戶名，加入回應裡
    hello = {
        "type": "text",
        "text": "哈囉！很感謝" + str(user_profile.display_name) + "加我為朋友。我知道的東西或許不比你多，但我認識各種職業，在找工作上，可以幫你的忙"
    }
    result_message_array.insert(0, TextSendMessage.new_from_json_dict(hello))
        
    # 創建個人資料表 #增加個人ID
    user_ID = user_profile.display_name
    user_data = {
        "user_ID": user_ID,
        "character": [],
        "finding_job": [],
        "good_cv" : [],
        "good_pic" : [],
        "good_con" : [],
        "satisfy_job": [],
        "curious_sal" :[],
        "menu" : []
    }
    
    dn = "user_message/"
    if not os.path.exists(dn):
        os.makedirs(dn)
    fn = str(user_ID) + ".json"
    f = open(dn + fn, "w", encoding="utf-8")
    json.dump(user_data, f)
    f.close()
    
    # 消息發送
    line_bot_api.reply_message(
        event.reply_token,
        result_message_array
    )


In [7]:
'''

handler處理文字消息

收到用戶回應的文字消息，
按文字消息內容，往素材資料夾中，找尋以該內容命名的資料夾，讀取裡面的reply.json

轉譯json後，將消息回傳給用戶

'''

# 引用Line套件
from linebot.models import (
    MessageEvent, TextMessage)

# 定義斷詞功能
def wordcut(s):
    s = s.replace("\r", "").replace("\n", "")
    return " ".join(cut(s))

keepgoing_f1 = ''
keepgoing_f2 = ''

# 文字消息處理
@handler.add(MessageEvent,message=TextMessage)
def process_text_message(event):
    global keepgoing_f1
    global keepgoing_f2
    exclude_list = ['在職工作者', '應屆畢業生', '學生', '不告訴你', '很滿意', '還行', '不太滿意', '挺好奇的', '有一點', '不會',
                    '自行探索', '找工作中', '還不急', '沒有', '一定可以', '希望...可以', '不行', '完美', '不敢保證',  '沒什麼信心',
                    '有自信', '還好', '不太有', '5G', '5g', '中美貿易', '企業表現', '智慧產業', '隨機搜尋', '我了解了', '開始！', '面試結束',
                    '重新面試', '觀看結果']
    
    if event.message.text in exclude_list:
        pass
    
    elif event.message.text == '取消':
        line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='請繼續使用其他功能')
        )

    # 如果詢問功能建議，針對個人資料進行計分，提供建議
    elif event.message.text == '有建議嗎':
        user_profile = line_bot_api.get_profile(event.source.user_id)
        fn = user_profile.display_name
        dn = "user_message/" + fn + ".json"
        with open(dn, "r") as f:
            data = json.load(f)
        
        fun1_score = 0  # 描述搜尋
        fun2_score = 0  # 產業新聞
        fun3_score = 0  # 薪資預測
        fun4_score = 0  # 職缺氣象
        fun5_score = 0  # 相由心生
        fun6_score = 0  # 面試捕手
        
        # 目前身分
        if data['character'] == []:
            pass
        elif data['character'][-1] == '在職工作者':
            fun2_score += 1
        elif data['character'][-1] == '應屆畢業生':
            fun1_score += 1
            fun3_score += 1
            fun4_score += 1
            fun5_score += 1
            fun6_score += 1
        elif data['character'][-1] == '學生':
            fun1_score += 1
        
        # 找工作中
        if data['finding_job'] == []:
            pass
        elif data['finding_job'][-1] == '找工作中':
            fun1_score += 1
        elif data['finding_job'][-1] == '還不急':
            fun1_score += 0.5
        else:
            pass
        
        # 履歷信心
        if data['good_cv'] == []:
            pass
        elif data['good_cv'][-1] == '希望...可以':
            fun4_score += 0.5
        elif data['good_cv'][-1] == '不行':
            fun4_score += 1
        else:
            pass
        
        # 照片信心
        if data['good_pic'] == []:
            pass
        elif data['good_pic'][-1] == '不敢保證':
            fun5_score += 0.5
        elif data['good_pic'][-1] == '沒什麼信心':
            fun5_score += 1
        else:
            pass
        
        # 交談信心
        if data['good_con'] == []:
            pass
        elif data['good_con'][-1] == '還好':
            fun6_score += 0.5
        elif data['good_con'][-1] == '不太有':
            fun6_score += 1
        
        # 滿意工作
        if data['satisfy_job'] == []:
            pass
        elif data['satisfy_job'][-1] == '還行':
            fun1_score += 0.5
        elif data['satisfy_job'][-1] == '不太滿意':
            fun1_score += 1
        
        # 好奇薪資
        if data['curious_sal'] == []:
            pass
        elif data['curious_sal'][-1] == '挺好奇的':
            fun3_score += 1
        elif data['curious_sal'][-1] == '有一點':
            fun3_score += 0.5
        
        score_list = [('描述搜尋', fun1_score), ('產業新聞', fun2_score), ('薪資預測', fun3_score), ('職缺氣象', fun4_score), ('相由心生', fun5_score), ('面試捕手', fun6_score)]
        score_sorted = sorted(score_list, key=lambda x : x[1], reverse=True)
        
        suggestion = {
        "type": "text",
        "text": "根據您的情況，建議你可先使用以下功能\n1." + score_sorted[0][0] + "\n2." + score_sorted[1][0] + "\n3." + score_sorted[2][0]
        }
        
        result_message_array = TextSendMessage.new_from_json_dict(suggestion)
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
        
    elif event.message.text == '描述搜尋':
        keepgoing_f1 = None
        keepgoing_f2 = None
        keepgoing_f1 = 'yes'
        
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text='請隨意輸入您想像中的工作內容、您的專長、技能、甚至是個人興趣皆可，我將給您職缺搜尋的建議。例如：教練，我想打程式')
        )
        
    elif keepgoing_f1 == 'yes':
        keepgoing_f2 = None
        keepgoing_f1 = None
        label = ["其他專業", "客服支援", "教育傳播","管理財經","行銷業務","製造工程"]
        # 引用MLP模型
        with open('./models/text_model2(addinfo_controlsample)/big2.pickle', 'rb') as f:
            big2 =pickle.load(f)
        with open('./models/text_model2(addinfo_controlsample)/big2_vec.pickle', 'rb') as f:
            vec = pickle.load(f)
        #with open('./models/text_model1/big1.pickle', 'rb') as f:
        #    big2 =pickle.load(f)
        #with open('./models/text_model1/big1_vec.pickle', 'rb') as f:
        #    vec = pickle.load(f)
        c = wordcut(event.message.text)
        v = vec.transform([c])
        j = label[big2.predict(v)[0]]
        if np.max(big2.predict_proba(v)[0]) >= 0.5:
        
            if j == "製造工程":
                label1 = ["生產管理","工程研發","製圖測量","營建施作","營建規劃","MIS網管",
                         "品管安規","軟體工程","製程規劃","環境衛生","生技研發","化工研發"]
                with open('./models/text_model2(addinfo_controlsample)/mid_製造工程.pickle', 'rb') as f:
                    mid1 = pickle.load(f)
                with open('./models/text_model2(addinfo_controlsample)/mid_製造工程_vec.pickle', 'rb') as f:
                    vec1 = pickle.load(f)
                #with open('./models/text_model1/mid_製造工程.pickle', 'rb') as f:
                #    mid1 = pickle.load(f)
                #with open('./models/text_model1/mid_製造工程_vec.pickle', 'rb') as f:
                #    vec1 = pickle.load(f)
                v = vec1.transform([c])
                line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='您的描述像是「' + j + '」中的「' + label1[mid1.predict(v)[0]] + '」，建議您可搜尋看看此職業類型的職缺')
                )

            elif j == "其他專業":
                label1 = ["操作技術","醫療專業","餐飲專業","運輸物流","採購倉管","軍警消防",
                          "醫療保健","保全警衛","其他專業","農林漁牧"]
                with open('./models/text_model2(addinfo_controlsample)/mid_其他專業.pickle', 'rb') as f:
                    mid2 = pickle.load(f)
                with open('./models/text_model2(addinfo_controlsample)/mid_其他專業_vec.pickle', 'rb') as f:
                    vec2 = pickle.load(f)
                #with open('./models/text_model1/mid_其他專業.pickle', 'rb') as f:
                #    mid2 = pickle.load(f)
                #with open('./models/text_model1/mid_其他專業_vec.pickle', 'rb') as f:
                #    vec2 = pickle.load(f)
                v = vec2.transform([c])
                line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='您的描述像是「' + j + '」中的「' + label1[mid2.predict(v)[0]] + '」，建議您可搜尋看看此職業類型的職缺')
                )

            elif j == "客服支援":
                label1 = ["美容美髮","旅遊休閒","維修服務","客戶服務"]
                with open('./models/text_model2(addinfo_controlsample)/mid_客服支援.pickle', 'rb') as f:
                    mid3 = pickle.load(f)
                with open('./models/text_model2(addinfo_controlsample)/mid_客服支援_vec.pickle', 'rb') as f:
                    vec3 = pickle.load(f)
                #with open('./models/text_model1/mid_客服支援.pickle', 'rb') as f:
                #    mid3 = pickle.load(f)
                #with open('./models/text_model1/mid_客服支援_vec.pickle', 'rb') as f:
                #    vec3 = pickle.load(f)
                v = vec3.transform([c])
                line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='您的描述像是「' + j + '」中的「' + label1[mid3.predict(v)[0]] + '」，建議您可搜尋看看此職業類型的職缺')
                )

            elif j == "教育傳播":
                label1 = ["教育輔導","文字編譯","藝術設計","學術研究","傳播藝術","傳媒採訪"]
                with open('./models/text_model2(addinfo_controlsample)/mid_教育傳播.pickle', 'rb') as f:
                    mid4 = pickle.load(f)
                with open('./models/text_model2(addinfo_controlsample)/mid_教育傳播_vec.pickle', 'rb') as f:
                    vec4 = pickle.load(f)
                #with open('./models/text_model1/mid_教育傳播.pickle', 'rb') as f:
                #    mid4 = pickle.load(f)
                #with open('./models/text)model1/mid_教育傳播_vec.pickle', 'rb') as f:
                #    vec4 = pickle.load(f)
                v = vec4.transform([c])
                line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='您的描述像是「' + j + '」中的「' + label1[mid4.predict(v)[0]] + '」，建議您可搜尋看看此職業類型的職缺')
                )
            elif j == "管理財經":
                label1 = ["金融專業","行政總務","財會稅務","法務智財","人力資源","經營幕僚"]
                with open('./models/text_model2(addinfo_controlsample)/mid_管理財經.pickle', 'rb') as f:
                    mid5 = pickle.load(f)
                with open('./models/text_model2(addinfo_controlsample)/mid_管理財經_vec.pickle', 'rb') as f:
                    vec5 = pickle.load(f)
                #with open('./models/text_model1/mid_管理財經.pickle', 'rb') as f:
                #    mid5 = pickle.load(f)
                #with open('./models/text_model1/mid_管理財經_vec.pickle', 'rb') as f:
                #    vec5 = pickle.load(f)
                v = vec5.transform([c])
                line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='您的描述像是「' + j + '」中的「' + label1[mid5.predict(v)[0]] + '」，建議您可搜尋看看此職業類型的職缺')
                )
            elif j == "行銷業務":
                label1 = ["業務銷售","行銷廣告","門市管理","產品企劃","貿易船務","專案管理"]
                with open('./models/text_model2(addinfo_controlsample)/mid_行銷業務.pickle', 'rb') as f:
                    mid6 = pickle.load(f)
                with open('./models/text_model2(addinfo_controlsample)/mid_行銷業務_vec.pickle', 'rb') as f:
                    vec6 = pickle.load(f)
                #with open('./models/text_model1/mid_行銷業務.pickle', 'rb') as f:
                #    mid6 = pickle.load(f)
                #with open('./models/text_model1/mid_行銷業務_vec.pickle', 'rb') as f:
                #    vec6 = pickle.load(f)
                v = vec6.transform([c])
                line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='您的描述像是「' + j + '」中的「' + label1[mid6.predict(v)[0]] + '」，建議您可搜尋看看此職業類型的職缺')
                )
        else:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='資訊不足（無法匹配相應職類），請輸入更多資訊')
            )
        
    elif event.message.text == '薪資預測':
        keepgoing_f1 = None
#        keepgoing_f2 = 'yes'
        
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text='請貼上您想查詢的「職缺網址」，目前支援：104、1111兩個網站')
        )
    
    elif re.match('https://', event.message.text):
    
#     elif keepgoing_f2 == 'yes':
#         keepgoing_f2 = None
        joblink = event.message.text

        pdcols = [
            'source',
            'category_top',
            'management',
            'expatriate',
            'work_time',
            'salary_max',
            'salary_min',
            'accept_experience',
            'education_min',
            'candidates',
            'industrial_top',
            'industrial_mid',
            'area'
        ]
        basedf = pd.DataFrame(columns=pdcols)

        if re.match("https://www.104.com.tw", joblink):
            job_dict = crawlerJob104(joblink)
            if job_dict:
                basedf = etl_104(basedf, job_dict)
                if job_dict['公司連結']:
                    company_dict = crawlerCompany104(job_dict['公司連結'])
                    industry = get_industry('104', company_dict['產業類型'])
                    basedf['industrial_top'] = pd.Series(industry[0])
                    basedf['industrial_mid'] = pd.Series(industry[1])

            else:
                line_bot_api.reply_message(
                    event.reply_token,
                    TextSendMessage(text='此網址無效，檢查看看是否已過徵才時間？')
                )
                
        elif re.match("https://m.104.com.tw", joblink):
            joblink = "https://www.104.com.tw/job/" + str(joblink.split('?')[0].split('/')[-1])
            job_dict = crawlerJob104(joblink)
            if job_dict:
                basedf = etl_104(basedf, job_dict)
                if job_dict['公司連結']:
                    company_dict = crawlerCompany104(job_dict['公司連結'])
                    industry = get_industry('104', company_dict['產業類型'])
                    basedf['industrial_top'] = pd.Series(industry[0])
                    basedf['industrial_mid'] = pd.Series(industry[1])

            else:
                line_bot_api.reply_message(
                    event.reply_token,
                    TextSendMessage(text='此網址無效，檢查看看是否已過徵才時間？')
                )

        elif re.match("https://www.1111.com.tw", joblink):
            job_dict = crawlerJob1111(joblink)
            if job_dict:
                basedf = etl_1111(basedf, job_dict)
                if job_dict['公司連結']:
                    company_dict = crawlerCompany1111(job_dict['公司連結'])
                    industry = get_industry('1111', company_dict['行業別'])
                    basedf['industrial_top'] = pd.Series(industry[0])
                    basedf['industrial_mid'] = pd.Series(industry[1])

            else:
                line_bot_api.reply_message(
                    event.reply_token,
                    TextSendMessage(text='此網址無效，檢查看看是否已過徵才時間？')
                )
                
        else:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='不好意思，無法處理104、1111以外的資訊')
            )
            
            
        
        if basedf.loc[0, 'salary_min'] == 0:
            
         
            # 資料與現有合併，方便dummy
            train_1 = pd.read_csv('./data/train_1.csv', encoding='utf-8')
            train_1 = pd.concat([basedf, train_1], ignore_index=True)
            train_1.drop(['salary_max', 'salary_min'], axis=1, inplace=True)

            # 開始製作迴歸模型所需data
            # 將多類別型欄位one hot encoding
            dummy = pd.get_dummies(train_1['category_top'])
            train_1 = pd.concat([train_1, dummy], axis=1)
            dummy = pd.get_dummies(train_1['work_time'])
            train_1 = pd.concat([train_1, dummy], axis=1)
            dummy = pd.get_dummies(train_1['education_min'])
            train_1 = pd.concat([train_1, dummy], axis=1)
            dummy = pd.get_dummies(train_1['candidates'])
            train_1 = pd.concat([train_1, dummy], axis=1)
            dummy = pd.get_dummies(train_1['area'])
            train_1 = pd.concat([train_1, dummy], axis=1)

            # 將二元類別欄位轉0 1
            train_1['source'].replace({104:0, 1111:1,'104':0, '1111':1}, inplace=True)
            train_1['management'].replace({'No':0, 'Yes':1}, inplace=True)
            train_1['expatriate'].replace({'No':0, 'Yes':1}, inplace=True)

            # 將原始欄位刪去
            train_1.drop(['category_top','work_time','education_min','candidates','area'], axis=1, inplace=True)

            # 共有五種產業大類df：民生服務、傳產製造、資訊科技、工商服務、文教傳播
            # 將各產業df中的industrial_mid one hot encoding
            if train_1.loc[0,'industrial_top'] == '民生服務':
                # 載入模型
                with open('./models/ind_1_salary_max.pickle', 'rb') as f:
                    ind_1_salary_max = pickle.load(f)
                with open('./models/ind_1_salary_min.pickle', 'rb') as f:
                    ind_1_salary_min = pickle.load(f)

                # 製作資料
                ind_1 = train_1.loc[train_1['industrial_top'] == '民生服務']
                dummy = pd.get_dummies(ind_1['industrial_mid'])
                ind_1 = pd.concat([ind_1, dummy], axis=1)
                ind_1.drop(['industrial_top','industrial_mid'], axis=1, inplace=True)

                pre_max = ind_1_salary_max.predict(ind_1)
                pre_max = np.around(pre_max[0],-3)

                pre_min = ind_1_salary_min.predict(ind_1)
                pre_min = np.around(pre_min[0],-3)

                if pre_max > pre_min:
                    line_bot_api.reply_message(
                        event.reply_token,
                        TextSendMessage(text='根據網址所提供的條件，我估計這個工作最低月薪為' + str(pre_min) + '元，準確率為88.78%。此外，我估計最高月薪為' + str(pre_max) + '元，準確率為74.71%。這些估計不包含福利、年終，數值也僅供參考，仍須視您的能力、資歷及公司薪酬架構安排而定')
                    )

                else:
                    line_bot_api.reply_message(
                        event.reply_token,
                        TextSendMessage(text='根據網址所提供的條件，我估計這個工作最低月薪資為' + str(pre_min) + '元，準確率為88.78%。最高薪資我則無法判別，不好意思')
                    )

            elif train_1.loc[0,'industrial_top']  == '傳產製造':
                # 載入模型
                with open('./models/ind_2_salary_max.pickle', 'rb') as f:
                    ind_2_salary_max = pickle.load(f)
                with open('./models/ind_2_salary_min.pickle', 'rb') as f:
                    ind_2_salary_min = pickle.load(f)

                ind_2 = train_1.loc[train_1['industrial_top'] == '傳產製造']
                dummy = pd.get_dummies(ind_2['industrial_mid'])
                ind_2 = pd.concat([ind_2, dummy], axis=1)
                ind_2.drop(['industrial_top','industrial_mid'], axis=1, inplace=True)

                pre_max = ind_2_salary_max.predict(ind_2)
                pre_max = np.around(pre_max[0],-3)

                pre_min = ind_2_salary_min.predict(ind_2)
                pre_min = np.around(pre_min[0],-3)

                if pre_max > pre_min:
                    line_bot_api.reply_message(
                        event.reply_token,
                        TextSendMessage(text='根據網址所提供的條件，我估計這個工作最低月薪為' + str(pre_min) + '元，準確率為87.54%。此外，我估計最高月薪為' + str(pre_max) + '元，準確率為74.62%。這些估計不包含福利、年終，數值也僅供參考，仍須視您的能力、資歷及公司薪酬架構安排而定')
                    )

                else:
                    line_bot_api.reply_message(
                        event.reply_token,
                        TextSendMessage(text='根據網址所提供的條件，我估計這個工作最低月薪為' + str(pre_min) + '元，準確率為87.54%。最高薪資我則無法判別，不好意思')
                    )

            elif train_1.loc[0,'industrial_top']  == '資訊科技':
                # 載入模型
                with open('./models/ind_3_salary_max.pickle', 'rb') as f:
                    ind_3_salary_max = pickle.load(f)
                with open('./models/ind_3_salary_min.pickle', 'rb') as f:
                    ind_3_salary_min = pickle.load(f)

                ind_3 = train_1.loc[train_1['industrial_top'] == '資訊科技']
                dummy = pd.get_dummies(ind_3['industrial_mid'])
                ind_3 = pd.concat([ind_3, dummy], axis=1)
                ind_3.drop(['industrial_top','industrial_mid'], axis=1, inplace=True)

                pre_max = ind_3_salary_max.predict(ind_3)
                pre_max = np.around(pre_max[0],-3)

                pre_min = ind_3_salary_min.predict(ind_3)
                pre_min = np.around(pre_min[0],-3)

                if pre_max > pre_min:
                    line_bot_api.reply_message(
                        event.reply_token,
                        TextSendMessage(text='根據網址所提供的條件，我估計這個工作最低月薪為' + str(pre_min) + '元，準確率為86.63%。此外，我估計最高月薪為' + str(pre_max) + '元，準確率為73.68%。這些估計不包含福利、年終，數值也僅供參考，仍須視您的能力、資歷及公司薪酬架構安排而定')
                    )

                else:
                    line_bot_api.reply_message(
                        event.reply_token,
                        TextSendMessage(text='根據網址所提供的條件，我估計這個工作最低月薪為' + str(pre_min) + '元，準確率為86.63%。最高薪資我則無法判別，不好意思')
                    )

            elif train_1.loc[0,'industrial_top']  == '工商服務':
                # 載入模型
                with open('./models/ind_4_salary_max.pickle', 'rb') as f:
                    ind_4_salary_max = pickle.load(f)
                with open('./models/ind_4_salary_min.pickle', 'rb') as f:
                    ind_4_salary_min = pickle.load(f)

                ind_4 = train_1.loc[train_1['industrial_top'] == '工商服務']
                dummy = pd.get_dummies(ind_4['industrial_mid'])
                ind_4 = pd.concat([ind_4, dummy], axis=1)
                ind_4.drop(['industrial_top','industrial_mid'], axis=1, inplace=True)

                pre_max = ind_4_salary_max.predict(ind_4)
                pre_max = np.around(pre_max[0],-3)

                pre_min = ind_4_salary_min.predict(ind_4)
                pre_min = np.around(pre_min[0],-3)

                if pre_max > pre_min:
                    line_bot_api.reply_message(
                        event.reply_token,
                        TextSendMessage(text='根據網址所提供的條件，我估計這個工作最低月薪為' + str(pre_min) + '元，準確率為86.63%。此外，我估計最高月薪為' + str(pre_max) + '元，準確率為73.68%。這些估計不包含福利、年終，數值也僅供參考，仍須視您的能力、資歷及公司薪酬架構安排而定')
                    )

                else:
                    line_bot_api.reply_message(
                        event.reply_token,
                        TextSendMessage(text='根據網址所提供的條件，我估計這個工作最低月薪為' + str(pre_min) + '元，準確率為86.63%。最高薪資我則無法判別，不好意思')
                    )

            elif train_1.loc[0,'industrial_top']  == '文教傳播':
                # 載入模型
                with open('./models/ind_5_salary_max.pickle', 'rb') as f:
                    ind_5_salary_max = pickle.load(f)
                with open('./models/ind_5_salary_min.pickle', 'rb') as f:
                    ind_5_salary_min = pickle.load(f)

                ind_5 = train_1.loc[train_1['industrial_top'] == '文教傳播']
                dummy = pd.get_dummies(ind_5['industrial_mid'])
                ind_5 = pd.concat([ind_5, dummy], axis=1)
                ind_5.drop(['industrial_top','industrial_mid'], axis=1, inplace=True)

                pre_max = ind_5_salary_max.predict(ind_5)
                pre_max = np.around(pre_max[0],-3)

                pre_min = ind_5_salary_min.predict(ind_5)
                pre_min = np.around(pre_min[0],-3)

                if pre_max > pre_min:
                    line_bot_api.reply_message(
                        event.reply_token,
                        TextSendMessage(text='根據網址所提供的條件，我估計這個工作最低月薪為' + str(pre_min) + '元，準確率為91.11%。此外，我估計最高薪資為' + str(pre_max) + '元，準確率為81.41%。這些估計不包含福利、年終，數值也僅供參考，仍須視您的能力、資歷及公司薪酬架構安排而定')
                    )

                else:
                    line_bot_api.reply_message(
                        event.reply_token,
                        TextSendMessage(text='根據網址所提供的條件，我估計這個工作最低月薪為' + str(pre_min) + '元，準確率為91.11%。最高薪資我則無法判別，不好意思')
                    )
            
        else:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='網站上似乎有寫薪資範圍了，請您檢查一下')
            )

    
    else:    
        try:
            # 讀取本地檔案，並轉譯成消息
            result_message_array =[]
            replyJsonPath = "素材/"+event.message.text+"/reply.json"
            result_message_array = detect_json_array_to_new_message_array(replyJsonPath)

            # 發送
            line_bot_api.reply_message(
                event.reply_token,
                result_message_array
            )
        except:
            sorry = {
                "type": "text",
                "text": "我聽不懂這句話的意思，Sorry!"
            }
            result_message_array = TextSendMessage.new_from_json_dict(sorry)
            line_bot_api.reply_message(
                event.reply_token,
                result_message_array
            )


In [8]:
'''

handler處理Postback Event

載入功能選單與啟動特殊功能

解析postback的data，並按照data欄位判斷處理

現有八個欄位
character, finding_job, good_cv, good_pic, 
good_con, satisfy_job, curious_sal, menu

若folder欄位有值，則
    紀錄個人tag
    讀取其reply.json，轉譯成消息，並發送

若menu欄位有值，則
    紀錄個人tag
    讀取其rich_menu_id，並取得用戶id，將用戶與選單綁定
    讀取其reply.json，轉譯成消息，並發送

'''
from linebot.models import (
    PostbackEvent
)

from urllib.parse import parse_qs 

@handler.add(PostbackEvent)
def process_postback_event(event):
    
    user_profile = line_bot_api.get_profile(event.source.user_id)

    query_string_dict = parse_qs(event.postback.data)
    
    print(query_string_dict)
    if 'character' in query_string_dict:
    
        result_message_array =[]
        replyJsonPath = '素材/'+ query_string_dict.get('character')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
        
        fn = user_profile.display_name
        dn = "user_message/" + fn + ".json"
        
        with open(dn, "r") as f:
            data = json.load(f)
            data["character"].append(query_string_dict.get('tag')[0])
        
        with open(dn, "w", encoding="utf-8") as f:
            json.dump(data, f)
            
  
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
    
    elif 'satisfy_job' in query_string_dict:
    
        result_message_array =[]
        replyJsonPath = '素材/'+query_string_dict.get('satisfy_job')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
        
        fn = user_profile.display_name
        dn = "user_message/" + fn + ".json"
        
        with open(dn, "r") as f:
            data = json.load(f)
            data["satisfy_job"].append(query_string_dict.get('tag')[0])
        
        with open(dn, "w", encoding="utf-8") as f:
            json.dump(data, f)
            
  
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
    
    elif 'curious_sal' in query_string_dict:
        
        # 綁定功能選單
        linkRichMenuId = open("素材/"+query_string_dict.get('curious_sal')[0]+'/rich_menu_id', 'r').read()
        line_bot_api.link_rich_menu_to_user(event.source.user_id,linkRichMenuId)
        
        # 詢問是否要功能建議
        replyJsonPath = '素材/'+query_string_dict.get('curious_sal')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
        
        fn = user_profile.display_name
        dn = "user_message/" + fn + ".json"
        
        with open(dn, "r") as f:
            data = json.load(f)
            data["curious_sal"].append(query_string_dict.get('tag')[0])
        
        with open(dn, "w", encoding="utf-8") as f:
            json.dump(data, f) 
              
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
        
    elif 'finding_job' in query_string_dict:
    
        result_message_array =[]
        replyJsonPath = '素材/'+query_string_dict.get('finding_job')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
        
        fn = user_profile.display_name
        dn = "user_message/" + fn + ".json"
        
        with open(dn, "r") as f:
            data = json.load(f)
            data["finding_job"].append(query_string_dict.get('tag')[0])
        
        with open(dn, "w", encoding="utf-8") as f:
            json.dump(data, f)
            
  
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
    
    elif 'good_cv' in query_string_dict:
    
        result_message_array =[]
        replyJsonPath = '素材/'+query_string_dict.get('good_cv')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
        
        fn = user_profile.display_name
        dn = "user_message/" + fn + ".json"
        
        with open(dn, "r") as f:
            data = json.load(f)
            data["good_cv"].append(query_string_dict.get('tag')[0])
        
        with open(dn, "w", encoding="utf-8") as f:
            json.dump(data, f)
            
  
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
    
    elif 'good_pic' in query_string_dict:
    
        result_message_array =[]
        replyJsonPath = '素材/'+query_string_dict.get('good_pic')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
        
        fn = user_profile.display_name
        dn = "user_message/" + fn + ".json"
        
        with open(dn, "r") as f:
            data = json.load(f)
            data["good_pic"].append(query_string_dict.get('tag')[0])
        
        with open(dn, "w", encoding="utf-8") as f:
            json.dump(data, f)
            
  
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
    
    elif 'good_con' in query_string_dict:
    
        # 綁定功能選單
        linkRichMenuId = open("素材/"+query_string_dict.get('good_con')[0]+'/rich_menu_id', 'r').read()
        line_bot_api.link_rich_menu_to_user(event.source.user_id,linkRichMenuId)
        
        # 詢問是否要功能建議
        replyJsonPath = '素材/'+query_string_dict.get('good_con')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
        
        fn = user_profile.display_name
        dn = "user_message/" + fn + ".json"
        
        with open(dn, "r") as f:
            data = json.load(f)
            data["good_con"].append(query_string_dict.get('tag')[0])
        
        with open(dn, "w", encoding="utf-8") as f:
            json.dump(data, f) 
              
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
    elif 'news' in query_string_dict:
        label_dic = {'label_0':[88, 90, 169, 170, 249, 305, 320, 340, 570, 664, 665, 671, 715, 722, 749, 753, 774, 812, 933, 940, 968, 1100, 1136, 1137, 1153, 1301, 1378, 1436, 1476, 1479, 1759, 1896, 1912, 1954, 1970, 2034, 2049, 2065, 2066, 2076, 2077, 2078, 2079, 2112, 2273, 2490, 2501, 2657, 2672, 2709, 2827, 2831, 2850, 2886, 2887, 2888, 2890, 3021, 3163, 3358, 3360, 3416, 3437, 3478, 3491, 3504, 3560, 3636, 3768, 3784, 3809, 3834, 3886, 3894, 3918, 4025, 4033, 4036, 4169, 4223, 4371, 4394, 4405, 4417, 4418, 4454, 4476, 4478, 4498, 4671, 4689, 4787, 4792, 4843, 4945, 4975, 4991, 5014, 5016, 5099, 5106, 5176, 5244, 5249, 5364, 5383, 5609, 5659, 5661, 5698, 5960, 5968, 6065, 6244, 6286, 6369, 6396, 6446, 6459, 6462, 6510, 6536, 6541, 6545, 6549, 6572, 6629, 6647, 6900, 6954, 6963, 6982, 6984, 7284, 7342, 7525, 7535, 7589, 7620, 7644, 7659, 7669, 7712, 7721, 7724, 7740, 7742, 7747, 7754, 7755, 7759, 7760, 7764, 7772, 7786, 7787, 7796, 7871, 8016, 8057, 8114, 8210, 8254, 8311, 8315, 8316, 8317, 8323, 8329, 8331, 8416, 8422, 8438, 8452, 8512, 8519, 8539, 8541, 8564, 8567, 8569, 8598, 8753, 8766, 8830, 8866, 8871, 8873, 8989, 8992, 9006, 9034, 9053, 9068, 9104, 9143, 9298, 9426, 9465, 9506, 9511, 9553, 9602, 9603, 9604, 9605, 9612, 9635, 9642, 9645, 9655, 9659, 9663, 9675, 9678, 9684, 9751, 9759, 9776, 9870, 9879, 9926, 9998, 9999, 10096, 10100, 10137, 10145, 10181, 10199, 10218, 10279, 10344, 10384, 10425, 10475, 10477, 10485, 10486, 10488, 10492, 10494, 10519, 10521, 10525, 10526, 10538, 10541, 10557, 10561, 10563, 10564, 10568, 10583, 10599, 10607, 10617, 10624, 10641, 10670, 10685, 10733, 10801, 10806, 10810, 10811, 10812, 10818, 10830, 10844, 10847, 10848, 10931, 10937, 10940, 10944, 10945, 10947, 10948, 10949, 10950, 10951, 10953, 10954, 10960, 10961, 10964, 10966, 10967, 10969, 10972, 10974, 10977, 10995, 11007, 11019, 11020, 11068, 11070, 11071, 11082, 11083, 11084, 11085, 11087, 11088, 11090, 11092, 11093, 11099, 11122, 11123, 11126, 11127, 11144, 11150, 11153, 11178, 11184, 11187, 11194, 11196, 11201, 11209, 11226, 11227, 11228, 11235, 11254, 11255, 11258, 11259, 11260, 11262, 11268, 11270, 11272, 11274, 11275, 11276, 11343, 11344, 11345, 11349, 11351, 11354, 11356, 11358, 11362, 11364, 11367, 11368, 11369, 11370, 11372, 11373, 11376, 11378, 11379, 11380, 11388, 11389, 11390, 11400, 11404, 11405, 11407, 11408, 11409, 11411, 11412, 11413, 11414, 11415, 11416, 11417, 11421, 11422, 11427, 11429, 11438, 11462, 11471, 11481, 11512, 11513, 11535, 11541, 11543, 11544, 11594, 11617, 11623, 11645, 11646, 11647, 11684, 11701, 11710, 11714, 11721, 11724, 11726, 11757, 11758, 11759, 11766, 11774, 11786, 11803, 11811, 11821, 11828, 11833, 11850, 11852, 11854, 11861, 11862, 11863, 11864, 11865, 11866, 11884, 11888, 11894, 11897, 11903, 11906, 11916, 11926, 11934, 11938, 11939, 11943, 11944, 11945, 11949, 11950, 11952, 11955, 11956, 11961, 11962, 11964, 11966, 11980, 11984, 11985, 11987, 11988, 11991, 11999, 12001, 12004, 12005, 12007, 12008, 12009, 12010, 12012, 12021, 12022, 12023, 12025, 12026, 12027, 12028, 12030, 12033, 12036, 12037, 12040, 12041, 12042, 12047, 12048, 12052, 12054, 12055, 12057, 12060, 12062, 12063, 12064, 12068, 12072, 12073, 12078, 12089, 12090, 12095, 12097, 12106, 12115, 12123, 12131, 12132, 12134, 12144, 12145, 12148, 12152, 12154, 12156, 12157, 12159, 12160, 12166, 12173, 12178, 12179, 12196, 12197, 12211, 12214, 12216, 12220, 12224, 12225, 12227, 12228, 12229, 12230, 12231, 12233, 12234, 12235, 12241, 12244, 12269, 12270, 12297, 12302, 12303, 12305, 12306, 12309, 12311, 12312, 12315, 12317, 12336, 12339, 12360, 12438, 12439, 12440, 12441, 12444, 12446, 12448, 12452, 12454, 12457, 12462, 12468, 12469, 12472, 12476, 12478, 12489, 12494, 12496, 12497, 12533, 12556, 12564, 12571, 12577, 12579, 12583, 12597, 12601, 12622, 12627, 12629, 12630, 12636, 12641, 12660, 12674, 12679, 12680, 12681, 12683, 12684, 12685, 12693, 12703, 12704, 12705, 12706, 12707, 12708, 12709, 12710, 12711, 12712, 12713, 12715, 12716, 12717, 12718, 12719, 12720, 12721, 12722, 12723, 12724, 12725, 12726, 12727, 12728, 12729, 12756, 12762, 12773, 12777, 12785, 12791, 12792, 12819, 12841, 12842, 12869, 12870, 12871, 12881, 12882, 12886, 12895, 12899, 12923, 12945, 12947, 12972, 12978, 12984, 12992, 12996, 12998, 12999, 13000, 13001, 13002, 13005, 13009, 13011, 13028, 13030, 13033, 13034, 13048, 13050, 13056, 13075, 13076, 13100, 13101, 13113, 13115, 13134, 13139, 13142, 13144, 13145, 13146, 13148, 13155, 13157, 13159, 13181, 13222, 13223, 13226, 13227, 13230, 13243, 13255, 13281, 13282, 13295, 13299, 13300, 13304, 13311, 13318, 13346, 13355, 13368, 13370, 13397, 13398, 13400, 13408, 13420, 13424, 13431, 13432, 13454, 13455, 13465, 13467, 13469, 13471, 13483, 13489, 13490, 13491, 13492, 13497, 13502, 13505, 13511, 13522, 13529, 13530, 13531, 13532, 13534, 13535, 13536, 13537, 13539, 13541, 13560, 13570, 13581, 13588, 13596, 13604, 13620, 13632, 13638, 13640, 13641, 13642, 13645, 13646, 13663, 13665, 13781, 13792, 13798, 13817, 13827, 13830, 13833, 13838, 13839, 13840, 13852, 13855, 13857, 13877, 13900, 13901, 13911, 13913, 13914, 13917, 13918, 13919, 13937, 13939, 13940, 13948, 13968, 13972, 13979, 13981, 13982, 13986, 13992, 13998, 14001, 14004, 14006, 14016, 14019, 14020, 14031, 14038, 14072, 14078, 14084, 14094, 14095, 14098, 14101, 14104, 14135, 14136, 14137, 14147, 14153, 14154, 14155, 14159, 14164, 14183, 14184, 14187, 14189, 14205, 14207, 14210, 14213, 14218, 14221, 14233, 14235, 14239, 14240, 14252, 14254, 14255, 14256, 14262, 14266, 14267, 14275, 14277, 14279, 14282, 14288, 14293, 14294, 14295, 14298, 14299, 14303, 14304, 14310, 14313, 14316, 14320, 14322, 14323, 14324, 14336, 14351, 14372, 14375, 14382, 14395, 14407, 14413, 14415, 14427, 14430, 14431, 14433, 14434, 14435, 14436, 14437, 14438, 14439, 14442, 14443, 14453, 14455, 14465, 14466, 14467, 14471, 14474, 14480, 14483, 14488, 14489, 14513, 14519, 14520, 14528, 14635, 14636, 14644, 14706, 14712, 14718, 14755, 14769, 14773, 14799, 14806, 14807, 14818, 14828, 14829, 14830, 14831, 14853, 14866, 14867, 14868, 14875, 14879, 14880, 14881, 14888, 14889, 14890, 14891, 14892, 14896, 14898, 14900, 14901, 14902, 14922, 14929, 14936, 14945, 14950, 14955, 14961, 14962, 14967, 14968, 14969, 14978, 14990, 14994, 14995, 14996, 14997, 15003, 15007, 15008, 15013, 15015, 15017, 15018, 15019, 15020, 15021, 15022, 15023, 15025, 15026, 15027, 15029, 15031, 15032, 15033, 15034, 15035, 15037, 15039, 15040, 15041, 15042, 15045, 15046, 15048, 15049, 15050, 15051, 15052, 15053, 15054, 15055, 15056, 15058, 15059, 15061, 15062, 15063, 15064, 15065, 15067, 15069, 15070, 15071, 15072, 15073, 15076, 15077, 15078, 15079, 15080, 15081, 15082, 15083, 15084, 15085, 15087, 15088, 15089, 15090, 15091, 15098, 15099, 15100, 15129, 15156, 15157, 15169, 15175, 15181, 15195, 15198, 15200, 15201, 15202, 15203, 15204, 15205, 15206, 15208, 15209, 15212, 15213, 15214, 15215, 15216, 15218, 15222, 15242, 15259, 15268, 15274, 15289, 15292, 15306, 15317, 15321, 15329, 15330, 15348, 15349, 15350, 15351, 15352, 15353, 15355, 15357, 15358, 15359, 15380, 15385, 15386, 15391, 15417, 15419, 15455, 15456, 15461, 15462, 15464, 15467, 15476, 15504, 15508, 15510, 15511, 15519, 15520, 15521, 15524, 15527, 15529, 15533, 15549, 15554, 15558, 15559, 15562, 15575, 15576, 15578, 15603, 15605, 15606, 15625, 15627, 15632, 15636, 15643, 15644, 15645, 15646, 15647, 15657, 15659, 15660, 15669, 15679, 15680, 15681, 15688, 15694, 15737, 15739, 15741, 15742, 15744, 15745, 15746, 15747, 15748, 15749, 15750, 15751, 15752, 15753, 15755, 15756, 15757, 15758, 15759, 15760, 15763, 15766, 15768, 15771, 15773, 15781, 15788, 15794, 15795, 15796, 15809, 15827, 15836, 15837, 15838, 15851, 15852, 15866, 15871, 15898, 15899, 15900, 15902, 15903, 15904, 15905, 15906, 15907, 15908, 15909, 15913, 15914, 15917, 15922, 15927, 15935, 15936, 15937, 15938, 15948, 15951, 15952, 15966, 15970, 15994, 16002, 16050, 16051, 16065, 16066, 16069, 16071, 16072, 16075, 16076, 16077, 16078, 16079, 16081, 16082, 16083, 16084, 16085, 16089, 16093, 16095, 16097, 16100, 16105, 16106, 16107, 16108, 16109, 16119, 16154, 16155, 16165, 16176, 16191, 16193, 16196, 16204, 16211, 16226, 16231, 16235, 16249, 16257, 16258, 16261, 16267, 16278, 16279, 16283, 16287, 16302, 16308, 16310, 16322, 16323, 16327, 16331, 16332, 16333, 16334, 16335, 16336, 16339, 16345, 16347, 16348, 16355, 16390, 16391, 16396, 16410, 16423, 16435, 16453, 16468, 16475, 16482, 16485, 16486, 16488, 16505, 16509, 16510, 16511, 16512, 16513, 16524, 16525, 16527, 16537, 16553, 16586, 16595, 16598, 16606, 16607, 16610, 16614, 16615, 16619, 16669, 16672, 16676, 16681, 16684, 16714, 16725, 16738, 16751, 16752, 16764, 16772, 16773, 16775, 16776, 16779, 16780, 16782, 16792, 16799, 16801, 16808, 16810, 16910, 16932, 16938, 16940, 16942, 16953, 16954, 16955, 16956, 16957, 16977, 16984, 16985, 16986, 17004, 17007, 17008, 17012, 17032, 17033, 17038, 17041, 17043, 17063, 17064, 17065, 17066, 17070, 17071, 17072, 17077, 17078, 17079, 17080, 17095, 17101, 17118, 17121, 17122, 17125, 17129, 17130, 17131, 17143, 17145, 17148, 17154, 17156, 17164, 17167, 17174, 17185, 17197, 17201, 17208, 17209, 17212, 17242, 17251, 17264, 17266, 17279, 17284, 17300, 17303, 17304, 17306, 17307, 17308, 17309, 17310, 17312, 17331, 17349, 17375, 17381, 17386, 17390, 17400, 17401, 17424, 17425, 17432, 17444, 17449, 17450, 17451, 17461, 17477, 17480, 17497, 17500, 17512, 17513, 17514, 17522, 17528, 17545, 17563, 17577, 17582, 17583, 17584, 17587, 17588, 17596, 17603, 17609, 17615, 17619, 17650, 17654, 17661, 17662, 17663, 17664, 17665, 17666, 17667, 17672, 17681, 17682, 17690, 17691, 17707, 17714, 17718, 17725, 17765, 17780, 17781, 17785, 17797, 17799, 17801, 17802, 17803, 17804, 17805, 17809, 17820, 17838, 17839, 17845, 17861, 17873, 17887, 17890, 17893, 17899, 17907, 17943, 17944, 17945, 17951, 17952, 17953, 17954, 17955, 17960, 17996, 17997, 17998, 18001, 18002, 18004, 18026, 18027, 18036, 18038, 18044, 18046, 18056, 18092, 18111, 18124, 18127, 18128, 18133, 18159, 18160, 18168, 18173, 18186, 18192, 18218, 18219, 18226, 18234, 18235, 18236, 18237, 18238, 18239, 18240, 18241, 18242, 18243, 18246, 18247, 18248, 18249, 18250, 18251, 18254, 18256, 18257, 18258, 18259, 18260, 18261, 18262, 18264, 18265, 18268, 18270, 18271, 18272, 18279, 18280, 18283, 18284, 18285, 18286, 18287, 18288, 18289, 18290, 18291, 18293, 18294, 18295, 18296, 18297, 18298, 18299, 18300, 18301, 18302, 18304, 18305, 18306, 18307, 18308, 18312, 18317, 18318, 18319, 18320, 18321, 18322, 18323, 18325, 18326, 18327, 18328, 18329, 18330, 18332, 18333, 18335, 18336, 18337, 18338, 18339, 18340, 18341, 18345, 18346, 18347, 18349, 18350, 18355, 18356, 18358, 18360, 18365, 18376, 18378, 18383, 18387, 18389, 18390, 18392, 18393, 18396, 18400, 18401, 18402, 18403, 18405, 18406, 18408, 18413, 18415, 18420, 18425, 18426, 18427, 18430, 18431, 18432, 18434, 18435, 18437, 18439, 18443, 18444, 18445, 18446, 18447, 18450, 18453, 18464, 18465, 18466, 18467, 18468, 18469, 18470, 18472, 18473, 18474, 18475, 18476, 18477, 18478, 18479, 18480, 18482, 18483, 18486, 18490, 18494, 18498, 18504, 18505, 18508, 18509, 18510, 18512, 18513, 18515, 18516, 18518, 18520, 18522, 18523, 18525, 18526, 18527, 18528, 18529, 18530, 18531, 18532, 18533, 18534, 18535, 18536, 18539, 18540, 18541, 18542, 18543, 18545, 18547, 18548, 18549, 18550, 18551, 18552, 18553, 18554, 18555, 18556, 18558, 18559, 18560, 18561, 18563, 18566, 18568, 18571, 18572, 18575, 18576, 18577, 18578, 18581, 18582, 18583, 18584, 18585, 18586, 18587, 18590, 18598, 18600, 18601, 18605, 18606, 18611, 18612, 18613, 18614, 18615, 18618, 18619, 18622, 18623, 18625, 18626, 18627, 18637, 18638, 18645, 18652, 18657, 18658, 18659, 18660, 18661, 18662, 18670, 18671, 18678, 18707, 18719, 18721, 18723, 18743, 18745, 18748, 18778, 18780, 18785, 18786, 18796, 18797, 18803, 18806, 18819, 18833, 18841, 18858, 18866, 18869, 18871, 18883, 18913, 18918, 18919, 18920, 18930, 18941, 18942, 18943, 18944, 18945, 18947, 18954, 18956, 18962, 18963, 18965, 18969, 18973, 18975, 18979, 18980, 18981, 18990, 18996, 18997, 18998, 19002, 19006, 19011, 19012, 19013, 19033, 19043, 19044, 19050, 19073, 19074, 19076, 19077, 19083, 19084, 19085, 19087, 19088, 19089, 19091, 19092, 19093, 19094, 19095, 19096, 19097, 19099, 19100, 19101, 19102, 19103, 19104, 19106, 19108, 19114, 19115, 19116, 19117, 19118, 19121, 19124, 19126, 19127, 19132, 19133, 19134, 19135, 19136, 19137, 19138, 19139, 19140, 19141, 19142, 19143, 19144, 19146, 19147, 19148, 19149, 19150, 19151, 19152, 19153, 19154, 19156, 19157, 19158, 19159, 19160, 19161, 19162, 19163, 19164, 19166, 19167, 19168, 19169, 19170, 19171, 19172, 19173, 19174, 19175, 19176, 19178, 19179, 19180, 19181, 19182, 19183, 19184, 19185, 19186, 19187, 19188, 19189, 19190, 19191, 19192, 19193, 19194, 19195, 19197, 19199, 19203, 19204, 19205, 19206, 19207, 19208, 19210, 19211, 19212, 19214, 19218, 19219, 19220, 19224, 19225, 19226, 19227, 19228, 19230, 19231, 19232, 19233, 19236, 19238, 19239, 19240, 19242, 19243, 19244, 19245, 19247, 19248, 19249, 19250, 19251, 19252, 19253, 19254, 19255, 19256, 19257, 19258, 19259, 19260, 19261, 19262, 19263, 19266, 19268, 19269, 19270, 19271, 19272, 19273, 19274, 19275, 19277, 19278, 19279, 19280, 19281, 19282, 19283, 19285, 19286, 19287, 19289, 19290, 19291, 19292, 19294, 19295, 19296, 19297, 19298, 19299, 19300, 19301, 19302, 19303, 19304, 19305, 19308, 19309, 19310, 19311, 19312, 19313, 19314, 19315, 19317, 19319, 19320, 19321, 19322, 19323, 19324, 19325, 19326, 19327, 19329, 19330, 19331, 19332, 19333, 19334, 19336, 19337, 19338, 19339, 19341, 19342, 19343, 19344, 19345, 19347, 19349, 19350, 19351, 19352, 19355, 19356, 19358, 19359, 19360, 19391, 19393, 19413, 19414, 19415, 19419, 19421, 19423, 19426, 19429, 19434, 19437, 19439, 19440, 19442, 19445, 19446, 19449, 19450, 19462, 19475, 19480, 19500, 19502, 19510, 19518, 19522, 19537, 19542, 19543, 19546, 19548, 19549, 19565, 19568, 19590, 19605, 19606, 19610, 19612, 19613, 19616, 19617, 19621, 19622, 19623, 19626, 19627, 19629, 19630, 19631, 19633, 19635, 19636, 19641, 19647, 19648, 19649, 19650, 19651, 19653, 19654, 19658, 19659, 19660, 19661, 19662, 19664, 19665, 19666, 19667, 19668, 19669, 19671, 19673, 19674, 19675, 19676, 19677, 19678, 19679, 19680, 19684, 19685, 19687, 19688, 19689, 19695, 19700, 19701, 19703, 19708, 19709, 19713, 19714, 19715, 19716, 19718, 19720, 19725, 19733, 19736, 19739, 19740, 19741, 19743, 19744, 19746, 19751, 19752, 19753, 19759, 19761, 19762, 19764, 19767, 19770, 19771, 19772, 19773, 19774, 19775, 19777, 19778, 19779, 19780, 19781, 19782, 19783, 19785, 19786, 19788, 19789, 19790, 19791, 19793, 19794, 19797, 19798, 19801, 19802, 19803, 19804, 19805, 19806, 19807, 19808, 19809, 19810, 19812, 19815, 19819, 19822, 19823, 19824, 19825, 19826, 19831, 19832, 19833, 19836, 19838, 19839, 19840, 19841, 19842, 19843, 19844, 19847, 19848, 19850, 19851, 19852, 19860, 19861, 19862, 19863, 19864, 19865, 19866, 19867, 19868, 19869, 19872, 19876, 19877, 19878, 19879, 19880, 19882, 19884, 19885, 19887, 19889, 19890, 19891, 19892, 19894, 19901, 19906, 19907, 19908, 19910, 19911, 19912, 19914, 19915, 19916, 19917, 19918, 19919, 19920, 19922, 19923, 19924, 19925, 19928, 19929, 19930, 19933, 19935, 19945, 19952, 19976, 19985, 20006, 20015, 20016, 20018, 20019, 20022, 20028, 20043, 20052, 20058, 20059, 20063, 20071, 20089, 20105, 20106, 20109, 20119, 20123, 20124, 20125, 20143, 20144, 20153, 20157, 20159, 20164, 20167, 20171, 20172, 20173, 20176, 20178, 20180, 20182, 20183, 20189, 20193, 20194, 20203, 20204, 20218, 20219, 20220, 20221, 20222, 20225, 20226, 20227, 20228, 20229, 20230, 20232, 20235, 20239, 20247, 20249, 20251, 20252, 20253, 20257, 20258, 20259, 20263, 20264, 20278, 20279, 20289, 20293, 20321, 20322, 20325, 20326, 20328, 20346, 20352, 20379, 20386, 20391, 20402, 20403, 20405, 20408, 20422, 20424, 20425, 20428, 20443, 20445, 20446, 20447, 20462, 20463, 20473, 20499, 20502, 20513, 20520, 20521, 20522, 20523, 20525, 20528, 20541, 20545, 20550, 20551, 20559, 20569, 20570, 20572, 20586, 20627, 20628, 20634, 20644, 20645, 20646, 20654, 20657, 20659, 20661, 20664, 20668, 20674, 20676, 20685, 20706, 20709, 20727, 20733, 20735, 20757, 20811, 20812, 20839, 20844, 20845, 20852, 20857, 20864, 20865, 20868, 20870, 20876, 20878, 20881, 20882, 20888, 20889, 20890, 20900, 20901, 20902, 20904, 20906, 20912, 20924, 20927, 20933, 20935, 20937, 20949, 20953, 20957, 20964, 20966, 20967, 20977, 20989, 20995, 21000, 21003, 21004, 21007, 21011, 21012, 21013, 21015, 21016, 21017, 21023, 21024, 21025, 21026, 21027, 21028, 21029, 21031, 21033, 21034, 21039, 21040, 21041, 21043, 21044, 21046, 21047, 21048, 21049, 21052, 21054, 21057, 21059, 21060, 21062, 21064, 21066, 21070, 21073, 21074, 21075, 21078, 21079, 21081, 21082, 21083, 21084, 21085, 21088, 21089, 21092, 21093, 21095, 21096, 21097, 21099, 21102, 21106, 21107, 21111, 21112, 21114, 21119, 21123, 21124, 21125, 21126, 21127, 21130, 21131, 21134, 21136, 21141, 21145, 21146, 21147, 21148, 21149, 21150, 21180, 21190, 21198, 21202, 21207, 21226, 21246, 21280, 21287, 21316, 21332, 21336, 21338, 21340, 21347, 21353, 21360, 21382, 21386, 21390, 21398, 21423, 21427, 21434, 21435, 21436, 21439, 21441, 21443, 21445, 21465, 21466, 21486, 21511, 21517, 21518, 21520, 21543, 21553, 21560, 21561, 21563, 21573, 21574, 21575, 21581, 21584, 21586, 21596, 21608, 21629, 21635, 21636, 21644, 21645, 21647, 21662, 21687, 21697, 21702, 21704, 21707, 21708, 21709, 21714, 21727, 21729, 21734, 21741, 21751, 21754, 21756, 21757, 21760, 21774, 21780],
                     'label_1':[4, 30, 36, 66, 71, 73, 74, 76, 77, 79, 81, 85, 89, 93, 100, 102, 114, 115, 136, 137, 157, 159, 160, 163, 177, 178, 184, 193, 214, 215, 224, 228, 235, 271, 284, 286, 287, 288, 289, 290, 291, 292, 299, 303, 321, 324, 334, 341, 342, 350, 359, 360, 361, 362, 364, 367, 371, 377, 380, 382, 385, 390, 408, 423, 424, 426, 431, 433, 436, 437, 450, 452, 455, 489, 491, 499, 507, 508, 509, 517, 524, 551, 555, 560, 561, 562, 569, 583, 587, 589, 590, 593, 594, 596, 597, 601, 602, 604, 605, 606, 608, 610, 614, 615, 616, 619, 627, 628, 629, 646, 647, 651, 655, 657, 693, 694, 695, 696, 698, 699, 704, 705, 706, 707, 709, 723, 776, 779, 789, 793, 806, 807, 815, 816, 817, 818, 824, 825, 826, 830, 832, 839, 842, 844, 845, 847, 853, 868, 870, 878, 881, 901, 921, 928, 932, 935, 938, 942, 944, 952, 953, 955, 956, 965, 967, 975, 977, 980, 982, 1001, 1013, 1027, 1033, 1037, 1041, 1047, 1050, 1052, 1054, 1056, 1057, 1058, 1059, 1060, 1062, 1069, 1070, 1071, 1076, 1077, 1080, 1081, 1083, 1087, 1102, 1121, 1138, 1139, 1144, 1152, 1269, 1279, 1285, 1288, 1289, 1294, 1302, 1329, 1334, 1353, 1356, 1357, 1359, 1375, 1390, 1392, 1403, 1406, 1418, 1426, 1438, 1461, 1465, 1469, 1472, 1473, 1493, 1496, 1497, 1498, 1514, 1536, 1569, 1590, 1595, 1602, 1608, 1611, 1615, 1617, 1618, 1630, 1638, 1640, 1641, 1642, 1645, 1647, 1649, 1650, 1652, 1653, 1658, 1660, 1662, 1663, 1668, 1673, 1677, 1681, 1684, 1688, 1693, 1704, 1714, 1715, 1729, 1731, 1734, 1737, 1741, 1747, 1753, 1756, 1760, 1768, 1770, 1774, 1777, 1786, 1787, 1792, 1793, 1808, 1819, 1826, 1831, 1837, 1840, 1847, 1850, 1857, 1861, 1865, 1878, 1881, 1882, 1883, 1884, 1888, 1897, 1898, 1903, 1908, 1909, 1910, 1913, 1914, 1922, 1924, 1928, 1940, 1947, 1950, 1951, 1952, 1955, 1956, 1957, 1960, 1962, 1985, 1986, 1992, 2003, 2005, 2006, 2008, 2010, 2016, 2017, 2018, 2021, 2026, 2030, 2053, 2056, 2059, 2060, 2063, 2064, 2070, 2085, 2096, 2097, 2114, 2115, 2119, 2122, 2123, 2124, 2131, 2146, 2147, 2153, 2160, 2168, 2169, 2174, 2176, 2177, 2182, 2183, 2188, 2205, 2235, 2237, 2240, 2241, 2242, 2245, 2246, 2247, 2248, 2253, 2254, 2257, 2262, 2274, 2281, 2284, 2287, 2289, 2292, 2308, 2314, 2315, 2317, 2334, 2335, 2339, 2340, 2342, 2343, 2344, 2346, 2353, 2357, 2368, 2382, 2385, 2390, 2395, 2396, 2402, 2408, 2417, 2418, 2421, 2426, 2428, 2429, 2437, 2442, 2445, 2451, 2457, 2458, 2467, 2476, 2482, 2506, 2516, 2517, 2520, 2527, 2528, 2531, 2533, 2537, 2540, 2541, 2542, 2543, 2544, 2545, 2546, 2554, 2559, 2565, 2566, 2569, 2573, 2582, 2586, 2591, 2593, 2594, 2595, 2596, 2597, 2629, 2637, 2646, 2648, 2651, 2654, 2670, 2673, 2677, 2679, 2686, 2690, 2691, 2693, 2694, 2698, 2701, 2706, 2711, 2716, 2726, 2727, 2728, 2752, 2753, 2760, 2763, 2765, 2769, 2770, 2771, 2789, 2791, 2806, 2807, 2808, 2813, 2816, 2825, 2826, 2838, 2843, 2849, 2863, 2864, 2865, 2866, 2868, 2869, 2870, 2878, 2884, 2889, 2891, 2893, 2900, 2902, 2903, 2915, 2920, 2924, 2927, 2928, 2934, 2942, 2943, 2944, 2945, 2954, 2955, 2960, 2961, 2963, 2964, 2967, 2968, 2969, 2978, 2989, 2991, 2994, 2999, 3014, 3015, 3016, 3017, 3018, 3019, 3022, 3023, 3029, 3032, 3052, 3053, 3055, 3056, 3059, 3062, 3063, 3064, 3074, 3076, 3080, 3082, 3083, 3084, 3094, 3095, 3110, 3113, 3114, 3117, 3122, 3125, 3126, 3135, 3136, 3137, 3143, 3145, 3157, 3164, 3166, 3176, 3177, 3179, 3185, 3193, 3197, 3199, 3202, 3204, 3219, 3220, 3224, 3226, 3230, 3233, 3235, 3236, 3242, 3250, 3257, 3262, 3263, 3269, 3270, 3274, 3276, 3282, 3283, 3285, 3287, 3288, 3292, 3294, 3303, 3314, 3316, 3329, 3330, 3344, 3352, 3353, 3368, 3422, 3432, 3443, 3449, 3461, 3462, 3479, 3490, 3496, 3498, 3507, 3516, 3528, 3530, 3533, 3534, 3537, 3538, 3545, 3546, 3554, 3564, 3569, 3570, 3571, 3578, 3583, 3598, 3600, 3605, 3612, 3621, 3622, 3623, 3624, 3627, 3629, 3634, 3643, 3644, 3648, 3667, 3670, 3673, 3676, 3677, 3679, 3680, 3681, 3682, 3690, 3693, 3698, 3701, 3704, 3708, 3716, 3717, 3718, 3720, 3721, 3728, 3731, 3735, 3743, 3771, 3779, 3788, 3798, 3800, 3816, 3819, 3824, 3825, 3827, 3829, 3844, 3871, 3876, 3883, 3884, 3885, 3901, 3913, 3914, 3916, 3917, 3940, 3941, 3943, 3947, 3948, 3950, 3962, 3967, 3968, 3970, 3971, 3974, 3976, 3977, 3978, 3980, 3990, 4003, 4005, 4014, 4015, 4019, 4031, 4035, 4037, 4043, 4046, 4055, 4058, 4059, 4063, 4074, 4084, 4127, 4134, 4141, 4152, 4154, 4170, 4172, 4173, 4178, 4204, 4237, 4248, 4271, 4272, 4276, 4281, 4282, 4295, 4299, 4301, 4310, 4313, 4321, 4329, 4343, 4347, 4352, 4355, 4356, 4370, 4390, 4396, 4398, 4400, 4401, 4407, 4410, 4439, 4446, 4449, 4450, 4458, 4462, 4463, 4475, 4480, 4481, 4483, 4486, 4500, 4502, 4503, 4504, 4508, 4509, 4510, 4511, 4513, 4516, 4523, 4526, 4527, 4538, 4539, 4540, 4541, 4543, 4546, 4551, 4552, 4553, 4554, 4560, 4561, 4563, 4564, 4566, 4567, 4574, 4575, 4576, 4578, 4584, 4588, 4593, 4595, 4597, 4601, 4603, 4607, 4608, 4609, 4610, 4613, 4618, 4619, 4629, 4634, 4635, 4642, 4644, 4647, 4659, 4668, 4669, 4677, 4687, 4690, 4693, 4694, 4695, 4699, 4700, 4706, 4708, 4711, 4712, 4714, 4718, 4721, 4722, 4726, 4729, 4730, 4746, 4747, 4750, 4751, 4765, 4779, 4780, 4782, 4784, 4786, 4796, 4799, 4803, 4820, 4826, 4836, 4838, 4842, 4845, 4855, 4862, 4863, 4864, 4866, 4874, 4877, 4879, 4880, 4884, 4888, 4890, 4891, 4926, 4928, 4934, 4943, 4979, 5004, 5017, 5023, 5029, 5035, 5036, 5039, 5053, 5062, 5064, 5069, 5070, 5074, 5079, 5087, 5092, 5094, 5098, 5102, 5103, 5105, 5110, 5118, 5120, 5137, 5138, 5139, 5144, 5147, 5157, 5158, 5159, 5162, 5168, 5170, 5175, 5177, 5179, 5184, 5191, 5206, 5207, 5213, 5224, 5229, 5231, 5239, 5252, 5253, 5254, 5265, 5270, 5285, 5289, 5291, 5293, 5303, 5304, 5306, 5308, 5310, 5324, 5330, 5333, 5334, 5336, 5339, 5350, 5351, 5352, 5353, 5355, 5356, 5359, 5360, 5361, 5368, 5380, 5394, 5395, 5397, 5401, 5404, 5414, 5418, 5422, 5438, 5444, 5445, 5457, 5464, 5478, 5479, 5486, 5508, 5527, 5535, 5559, 5563, 5564, 5565, 5566, 5567, 5568, 5571, 5575, 5576, 5579, 5615, 5633, 5634, 5636, 5637, 5648, 5649, 5650, 5678, 5680, 5687, 5688, 5689, 5697, 5721, 5736, 5760, 5769, 5772, 5773, 5776, 5778, 5781, 5791, 5795, 5800, 5801, 5805, 5809, 5812, 5816, 5822, 5823, 5826, 5829, 5830, 5831, 5832, 5834, 5845, 5847, 5856, 5859, 5861, 5862, 5865, 5877, 5880, 5890, 5891, 5902, 5917, 5918, 5921, 5922, 5923, 5928, 5932, 5934, 5938, 5944, 5948, 5949, 5953, 5958, 5959, 5961, 5962, 5969, 5970, 5971, 5974, 5976, 5991, 5992, 5996, 5998, 5999, 6000, 6015, 6026, 6032, 6033, 6037, 6041, 6044, 6051, 6053, 6054, 6055, 6057, 6061, 6063, 6066, 6070, 6084, 6089, 6092, 6096, 6097, 6100, 6105, 6106, 6113, 6116, 6118, 6119, 6124, 6138, 6139, 6153, 6155, 6156, 6157, 6158, 6162, 6167, 6176, 6185, 6187, 6192, 6193, 6197, 6200, 6209, 6211, 6212, 6214, 6224, 6228, 6231, 6232, 6234, 6236, 6251, 6253, 6255, 6256, 6257, 6273, 6275, 6282, 6292, 6299, 6300, 6307, 6308, 6313, 6314, 6317, 6324, 6325, 6327, 6333, 6335, 6340, 6341, 6343, 6358, 6359, 6364, 6366, 6367, 6386, 6389, 6390, 6398, 6399, 6411, 6412, 6413, 6415, 6424, 6428, 6429, 6430, 6434, 6435, 6444, 6445, 6450, 6453, 6456, 6464, 6465, 6470, 6476, 6490, 6492, 6493, 6512, 6513, 6514, 6515, 6520, 6530, 6532, 6534, 6537, 6553, 6554, 6558, 6564, 6574, 6578, 6583, 6589, 6594, 6595, 6602, 6604, 6607, 6611, 6613, 6619, 6621, 6631, 6634, 6641, 6650, 6658, 6661, 6664, 6670, 6679, 6680, 6681, 6686, 6704, 6732, 6733, 6744, 6749, 6750, 6761, 6788, 6805, 6808, 6810, 6842, 6843, 6844, 6845, 6848, 6850, 6853, 6854, 6855, 6861, 6863, 6871, 6876, 6881, 6884, 6885, 6886, 6888, 6899, 6916, 6918, 6948, 6949, 6959, 6964, 6966, 6970, 6978, 6985, 6988, 6992, 7012, 7037, 7038, 7064, 7066, 7075, 7094, 7099, 7102, 7104, 7116, 7117, 7127, 7132, 7134, 7135, 7139, 7140, 7145, 7151, 7152, 7155, 7164, 7168, 7170, 7171, 7172, 7179, 7186, 7193, 7203, 7221, 7222, 7249, 7251, 7265, 7266, 7268, 7270, 7274, 7275, 7276, 7281, 7290, 7298, 7299, 7304, 7315, 7333, 7336, 7346, 7351, 7352, 7370, 7371, 7373, 7378, 7381, 7393, 7398, 7409, 7425, 7426, 7427, 7439, 7441, 7442, 7447, 7450, 7451, 7452, 7454, 7455, 7458, 7459, 7462, 7479, 7481, 7484, 7490, 7496, 7508, 7510, 7511, 7512, 7519, 7530, 7538, 7542, 7544, 7546, 7554, 7555, 7561, 7564, 7566, 7567, 7568, 7569, 7571, 7573, 7574, 7581, 7599, 7600, 7604, 7607, 7618, 7625, 7627, 7628, 7631, 7637, 7638, 7645, 7646, 7651, 7653, 7656, 7657, 7660, 7662, 7664, 7665, 7670, 7673, 7678, 7679, 7691, 7692, 7708, 7709, 7710, 7716, 7720, 7725, 7729, 7739, 7771, 7781, 7790, 7794, 7811, 7814, 7815, 7818, 7819, 7828, 7842, 7844, 7853, 7856, 7858, 7861, 7880, 7881, 7891, 7900, 7909, 7911, 7956, 7957, 7958, 7959, 7980, 8012, 8031, 8077, 8094, 8104, 8105, 8106, 8107, 8111, 8117, 8128, 8129, 8135, 8138, 8139, 8148, 8150, 8152, 8160, 8166, 8173, 8177, 8189, 8194, 8195, 8197, 8199, 8200, 8201, 8230, 8255, 8263, 8264, 8268, 8270, 8335, 8337, 8372, 8392, 8400, 8407, 8417, 8450, 8451, 8455, 8456, 8458, 8466, 8480, 8488, 8489, 8497, 8503, 8506, 8515, 8516, 8517, 8532, 8574, 8575, 8581, 8589, 8590, 8597, 8607, 8609, 8620, 8622, 8628, 8631, 8632, 8648, 8656, 8660, 8663, 8672, 8689, 8693, 8697, 8698, 8700, 8716, 8717, 8721, 8722, 8733, 8735, 8738, 8741, 8757, 8762, 8773, 8774, 8775, 8785, 8790, 8800, 8807, 8814, 8815, 8818, 8824, 8825, 8826, 8832, 8833, 8839, 8844, 8852, 8853, 8856, 8858, 8861, 8867, 8870, 8874, 8883, 8884, 8887, 8891, 8894, 8909, 8927, 8931, 8938, 8953, 8965, 8979, 8982, 8987, 8990, 8991, 9007, 9017, 9021, 9030, 9036, 9038, 9041, 9044, 9066, 9074, 9075, 9076, 9084, 9085, 9096, 9097, 9106, 9119, 9122, 9131, 9133, 9138, 9139, 9142, 9145, 9146, 9152, 9156, 9161, 9184, 9190, 9204, 9207, 9214, 9218, 9226, 9228, 9231, 9237, 9241, 9245, 9246, 9268, 9272, 9275, 9285, 9287, 9288, 9289, 9291, 9312, 9327, 9335, 9342, 9345, 9352, 9353, 9356, 9357, 9360, 9361, 9364, 9365, 9366, 9368, 9372, 9373, 9375, 9386, 9395, 9402, 9403, 9405, 9407, 9408, 9410, 9412, 9415, 9420, 9429, 9431, 9432, 9439, 9443, 9447, 9448, 9455, 9461, 9462, 9470, 9479, 9480, 9486, 9488, 9493, 9501, 9503, 9504, 9507, 9508, 9512, 9514, 9515, 9524, 9526, 9528, 9536, 9541, 9548, 9564, 9571, 9574, 9578, 9583, 9590, 9591, 9592, 9597, 9610, 9618, 9640, 9643, 9646, 9662, 9681, 9699, 9700, 9744, 9746, 9750, 9752, 9761, 9767, 9768, 9769, 9770, 9771, 9772, 9773, 9774, 9781, 9782, 9783, 9794, 9797, 9821, 9822, 9831, 9845, 9848, 9850, 9851, 9852, 9856, 9857, 9861, 9865, 9869, 9873, 9875, 9877, 9886, 9892, 9894, 9897, 9905, 9906, 9908, 9912, 9922, 9955, 9960, 9968, 9969, 9989, 9990, 9992, 10003, 10005, 10007, 10010, 10013, 10014, 10015, 10017, 10022, 10024, 10027, 10037, 10056, 10057, 10062, 10063, 10079, 10083, 10086, 10088, 10090, 10097, 10103, 10106, 10108, 10111, 10113, 10114, 10125, 10136, 10140, 10151, 10152, 10156, 10157, 10158, 10166, 10167, 10179, 10180, 10185, 10197, 10252, 10260, 10278, 10312, 10315, 10320, 10328, 10333, 10334, 10340, 10341, 10345, 10358, 10362, 10363, 10374, 10376, 10390, 10395, 10396, 10399, 10403, 10407, 10408, 10419, 10434, 10437, 10438, 10445, 10449, 10451, 10452, 10457, 10461, 10462, 10466, 10470, 10473, 10476, 10478, 10482, 10489, 10490, 10493, 10500, 10501, 10502, 10503, 10505, 10506, 10507, 10513, 10514, 10515, 10518, 10520, 10524, 10527, 10530, 10531, 10532, 10535, 10537, 10545, 10547, 10548, 10551, 10562, 10565, 10567, 10577, 10578, 10586, 10590, 10594, 10597, 10598, 10600, 10601, 10603, 10606, 10608, 10613, 10615, 10616, 10618, 10619, 10636, 10672, 10674, 10712, 10764, 10795, 10796, 10797, 10798, 10799, 10803, 10804, 10805, 10807, 10808, 10813, 10814, 10817, 10821, 10822, 10826, 10827, 10828, 10831, 10833, 10837, 10839, 10842, 10849, 10851, 10852, 10853, 10855, 10856, 10857, 10860, 10861, 10862, 10864, 10865, 10868, 10873, 10874, 10876, 10878, 10880, 10881, 10882, 10883, 10885, 10887, 10892, 10893, 10895, 10896, 10898, 10901, 10902, 10903, 10905, 10909, 10910, 10913, 10916, 10917, 10919, 10920, 10921, 10922, 10923, 10925, 10926, 10927, 10929, 10930, 10932, 10933, 10934, 10935, 10936, 10939, 10942, 10943, 10946, 10968, 10970, 10973, 10976, 10978, 10980, 10981, 10982, 10984, 10985, 10987, 10988, 10992, 11000, 11001, 11002, 11004, 11005, 11006, 11008, 11009, 11010, 11011, 11012, 11013, 11014, 11015, 11016, 11018, 11021, 11022, 11024, 11029, 11030, 11031, 11032, 11033, 11034, 11040, 11041, 11046, 11047, 11049, 11052, 11053, 11054, 11057, 11058, 11059, 11060, 11062, 11063, 11066, 11067, 11069, 11074, 11078, 11081, 11086, 11091, 11094, 11095, 11096, 11098, 11102, 11104, 11105, 11106, 11107, 11111, 11113, 11116, 11117, 11118, 11119, 11124, 11125, 11128, 11129, 11131, 11132, 11135, 11136, 11137, 11139, 11140, 11141, 11142, 11143, 11145, 11146, 11147, 11152, 11154, 11155, 11156, 11157, 11159, 11160, 11161, 11164, 11166, 11168, 11169, 11170, 11175, 11176, 11177, 11180, 11181, 11183, 11186, 11188, 11189, 11190, 11191, 11192, 11197, 11211, 11213, 11214, 11216, 11218, 11221, 11222, 11229, 11230, 11232, 11237, 11240, 11242, 11243, 11245, 11246, 11247, 11250, 11256, 11261, 11269, 11271, 11426, 11432, 11434, 11445, 11446, 11452, 11453, 11455, 11457, 11459, 11461, 11465, 11468, 11470, 11472, 11473, 11474, 11475, 11476, 11477, 11478, 11480, 11482, 11483, 11484, 11485, 11486, 11490, 11491, 11492, 11493, 11495, 11496, 11497, 11498, 11499, 11503, 11504, 11505, 11517, 11518, 11531, 11534, 11537, 11538, 11539, 11545, 11550, 11551, 11552, 11556, 11559, 11560, 11561, 11562, 11563, 11565, 11566, 11568, 11569, 11570, 11573, 11574, 11576, 11579, 11589, 11591, 11597, 11600, 11602, 11604, 11605, 11606, 11607, 11610, 11612, 11613, 11614, 11616, 11620, 11622, 11628, 11629, 11631, 11635, 11637, 11638, 11648, 11651, 11653, 11654, 11655, 11656, 11657, 11662, 11663, 11664, 11665, 11666, 11668, 11670, 11671, 11672, 11673, 11674, 11675, 11678, 11681, 11682, 11694, 11696, 11708, 11709, 11712, 11713, 11716, 11717, 11719, 11720, 11729, 11730, 11731, 11732, 11733, 11734, 11735, 11736, 11737, 11739, 11740, 11743, 11746, 11748, 11749, 11750, 11754, 11755, 11756, 11760, 11761, 11763, 11764, 11769, 11772, 11775, 11777, 11779, 11784, 11785, 11788, 11789, 11790, 11791, 11794, 11799, 11801, 11802, 11804, 11805, 11807, 11809, 11810, 11824, 11826, 11834, 11836, 11837, 11838, 11841, 11842, 11843, 11844, 11845, 11846, 11851, 11855, 11858, 11867, 11870, 11871, 11879, 11880, 11881, 11882, 11883, 11889, 11890, 11891, 11892, 11895, 11898, 11899, 11905, 11908, 11912, 11913, 11918, 11919, 11923, 11925, 11931, 11932, 11933, 11937, 11941, 11942, 11951, 11953, 11958, 11959, 11965, 11967, 11975, 11978, 11981, 11982, 11983, 11992, 11996, 12002, 12006, 12013, 12018, 12019, 12020, 12031, 12032, 12044, 12045, 12046, 12051, 12061, 12066, 12067, 12079, 12091, 12099, 12100, 12101, 12102, 12112, 12114, 12116, 12118, 12120, 12122, 12125, 12128, 12129, 12130, 12138, 12140, 12142, 12149, 12150, 12161, 12163, 12170, 12172, 12174, 12176, 12182, 12184, 12186, 12193, 12194, 12199, 12209, 12212, 12213, 12215, 12247, 12253, 12254, 12258, 12260, 12261, 12263, 12265, 12266, 12267, 12273, 12274, 12275, 12277, 12278, 12288, 12307, 12320, 12323, 12325, 12326, 12329, 12333, 12335, 12337, 12344, 12345, 12346, 12347, 12348, 12356, 12359, 12366, 12367, 12369, 12371, 12373, 12374, 12377, 12382, 12386, 12390, 12392, 12394, 12395, 12397, 12398, 12399, 12400, 12402, 12405, 12408, 12409, 12410, 12411, 12412, 12413, 12414, 12415, 12416, 12417, 12419, 12421, 12422, 12430, 12435, 12436, 12437, 12442, 12443, 12449, 12458, 12459, 12471, 12480, 12481, 12486, 12488, 12490, 12498, 12502, 12503, 12507, 12511, 12512, 12513, 12515, 12517, 12521, 12525, 12526, 12528, 12532, 12534, 12536, 12537, 12539, 12541, 12542, 12543, 12545, 12546, 12547, 12551, 12552, 12553, 12562, 12568, 12570, 12575, 12578, 12584, 12586, 12587, 12590, 12592, 12594, 12596, 12606, 12608, 12610, 12612, 12613, 12615, 12616, 12621, 12626, 12649, 12651, 12652, 12661, 12664, 12666, 12675, 12676, 12677, 12678, 12688, 12697, 12698, 12733, 12735, 12737, 12738, 12740, 12741, 12743, 12745, 12748, 12755, 12758, 12759, 12760, 12768, 12772, 12779, 12783, 12784, 12787, 12793, 12795, 12796, 12799, 12806, 12813, 12814, 12822, 12824, 12826, 12828, 12829, 12831, 12832, 12833, 12834, 12838, 12868, 12872, 12873, 12875, 12876, 12879, 12884, 12889, 12890, 12893, 12896, 12900, 12904, 12908, 12909, 12912, 12915, 12922, 12937, 12939, 12944, 12946, 12949, 12951, 12952, 12953, 12954, 12955, 12956, 12957, 12958, 12963, 12964, 12968, 12970, 12971, 12975, 12979, 12981, 12982, 12990, 12991, 12997, 13006, 13007, 13015, 13016, 13017, 13019, 13020, 13023, 13027, 13037, 13039, 13043, 13047, 13052, 13053, 13057, 13064, 13065, 13067, 13068, 13071, 13072, 13077, 13078, 13079, 13080, 13081, 13083, 13091, 13092, 13093, 13095, 13098, 13105, 13106, 13107, 13110, 13112, 13114, 13116, 13117, 13118, 13125, 13126, 13128, 13129, 13130, 13131, 13132, 13136, 13140, 13151, 13152, 13153, 13154, 13163, 13165, 13167, 13168, 13171, 13180, 13187, 13191, 13194, 13200, 13202, 13205, 13206, 13207, 13210, 13214, 13215, 13219, 13224, 13225, 13239, 13241, 13242, 13248, 13249, 13250, 13251, 13252, 13253, 13256, 13257, 13260, 13263, 13264, 13266, 13267, 13268, 13275, 13279, 13284, 13290, 13291, 13292, 13293, 13297, 13305, 13307, 13309, 13312, 13313, 13315, 13317, 13321, 13322, 13324, 13328, 13330, 13332, 13334, 13335, 13337, 13340, 13341, 13342, 13343, 13345, 13348, 13357, 13359, 13361, 13362, 13363, 13367, 13374, 13378, 13379, 13380, 13383, 13387, 13389, 13404, 13406, 13407, 13409, 13416, 13417, 13421, 13422, 13423, 13425, 13433, 13434, 13436, 13442, 13443, 13444, 13445, 13446, 13450, 13452, 13453, 13458, 13459, 13460, 13462, 13463, 13464, 13466, 13468, 13473, 13474, 13476, 13477, 13480, 13481, 13484, 13486, 13494, 13496, 13499, 13504, 13510, 13533, 13538, 13540, 13544, 13548, 13550, 13552, 13558, 13561, 13562, 13566, 13569, 13571, 13572, 13574, 13575, 13576, 13582, 13583, 13584, 13585, 13586, 13587, 13589, 13590, 13591, 13593, 13595, 13598, 13599, 13600, 13605, 13606, 13607, 13611, 13612, 13614, 13615, 13616, 13617, 13618, 13619, 13622, 13623, 13624, 13625, 13629, 13643, 13644, 13648, 13652, 13653, 13654, 13655, 13656, 13657, 13658, 13666, 13667, 13668, 13669, 13670, 13676, 13680, 13682, 13683, 13685, 13687, 13690, 13691, 13692, 13694, 13695, 13696, 13699, 13700, 13703, 13707, 13708, 13712, 13714, 13715, 13719, 13720, 13721, 13722, 13723, 13724, 13728, 13733, 13734, 13737, 13738, 13739, 13740, 13741, 13744, 13746, 13748, 13749, 13750, 13751, 13754, 13755, 13757, 13759, 13760, 13762, 13763, 13764, 13765, 13767, 13769, 13772, 13773, 13776, 13777, 13782, 13783, 13786, 13787, 13790, 13793, 13797, 13804, 13805, 13806, 13807, 13811, 13812, 13815, 13816, 13823, 13824, 13828, 13829, 13832, 13842, 13843, 13845, 13851, 13859, 13866, 13868, 13871, 13872, 13879, 13883, 13894, 13895, 13896, 13897, 13902, 13909, 13924, 13925, 13929, 13930, 13931, 13932, 13933, 13934, 13935, 13945, 13949, 13950, 13951, 13952, 13960, 13963, 13970, 13990, 14000, 14003, 14008, 14009, 14018, 14032, 14035, 14036, 14037, 14039, 14042, 14053, 14055, 14059, 14062, 14063, 14069, 14071, 14076, 14079, 14080, 14081, 14082, 14086, 14089, 14090, 14092, 14093, 14096, 14097, 14102, 14103, 14106, 14107, 14108, 14111, 14112, 14113, 14116, 14117, 14120, 14121, 14123, 14124, 14125, 14126, 14127, 14128, 14130, 14131, 14132, 14133, 14134, 14138, 14139, 14141, 14144, 14145, 14146, 14148, 14152, 14157, 14168, 14169, 14170, 14171, 14174, 14176, 14178, 14180, 14181, 14190, 14192, 14195, 14199, 14200, 14201, 14202, 14203, 14208, 14211, 14225, 14227, 14228, 14230, 14231, 14232, 14234, 14236, 14241, 14245, 14265, 14269, 14270, 14271, 14272, 14273, 14274, 14284, 14286, 14287, 14292, 14302, 14308, 14314, 14315, 14330, 14331, 14333, 14339, 14345, 14353, 14354, 14355, 14356, 14358, 14359, 14360, 14362, 14363, 14366, 14368, 14369, 14370, 14378, 14379, 14380, 14384, 14385, 14386, 14388, 14389, 14390, 14391, 14393, 14400, 14404, 14405, 14406, 14409, 14410, 14411, 14412, 14418, 14419, 14423, 14454, 14456, 14457, 14458, 14459, 14460, 14462, 14469, 14470, 14472, 14494, 14501, 14503, 14504, 14508, 14509, 14510, 14511, 14512, 14529, 14530, 14538, 14542, 14547, 14555, 14558, 14559, 14561, 14563, 14564, 14565, 14571, 14572, 14573, 14582, 14583, 14584, 14586, 14587, 14588, 14589, 14590, 14591, 14594, 14595, 14596, 14597, 14598, 14599, 14602, 14603, 14607, 14611, 14613, 14614, 14616, 14619, 14622, 14625, 14626, 14634, 14638, 14640, 14642, 14645, 14646, 14654, 14658, 14660, 14662, 14664, 14670, 14694, 14697, 14700, 14701, 14702, 14707, 14711, 14713, 14721, 14725, 14733, 14736, 14738, 14739, 14741, 14743, 14753, 14756, 14759, 14760, 14761, 14776, 14782, 14787, 14789, 14802, 14803, 14804, 14805, 14812, 14814, 14815, 14823, 14825, 14832, 14833, 14834, 14836, 14837, 14839, 14843, 14845, 14848, 14849, 14856, 14859, 14860, 14862, 14864, 14865, 14869, 14870, 14872, 14877, 14884, 14885, 14904, 14907, 14908, 14911, 14912, 14913, 14918, 14919, 14931, 14934, 14935, 14937, 14940, 14942, 14944, 14947, 14949, 14954, 14957, 14958, 14965, 14966, 14972, 14973, 14974, 14977, 14979, 14998, 15000, 15001, 15004, 15009, 15011, 15038, 15057, 15101, 15104, 15105, 15107, 15108, 15109, 15112, 15116, 15118, 15119, 15120, 15121, 15123, 15125, 15127, 15128, 15130, 15131, 15132, 15136, 15137, 15138, 15143, 15144, 15147, 15148, 15150, 15151, 15153, 15154, 15155, 15159, 15161, 15162, 15170, 15178, 15183, 15189, 15190, 15192, 15193, 15196, 15197, 15228, 15229, 15230, 15233, 15239, 15240, 15245, 15247, 15250, 15254, 15255, 15258, 15272, 15282, 15286, 15290, 15295, 15300, 15301, 15311, 15312, 15314, 15315, 15318, 15319, 15320, 15323, 15325, 15328, 15331, 15333, 15334, 15336, 15337, 15338, 15339, 15343, 15344, 15346, 15361, 15363, 15364, 15365, 15367, 15372, 15377, 15378, 15382, 15383, 15387, 15388, 15390, 15401, 15402, 15404, 15406, 15407, 15415, 15421, 15423, 15426, 15429, 15438, 15439, 15440, 15441, 15442, 15443, 15444, 15446, 15447, 15451, 15452, 15454, 15457, 15459, 15463, 15466, 15469, 15470, 15471, 15475, 15477, 15478, 15483, 15484, 15492, 15493, 15500, 15502, 15503, 15505, 15512, 15514, 15522, 15523, 15530, 15535, 15537, 15542, 15543, 15545, 15551, 15552, 15561, 15563, 15566, 15568, 15569, 15570, 15588, 15593, 15594, 15595, 15596, 15602, 15609, 15612, 15613, 15614, 15615, 15616, 15618, 15619, 15631, 15637, 15639, 15651, 15652, 15653, 15654, 15663, 15667, 15668, 15670, 15671, 15675, 15684, 15685, 15686, 15689, 15690, 15691, 15692, 15693, 15697, 15698, 15704, 15707, 15712, 15713, 15719, 15722, 15723, 15724, 15726, 15727, 15730, 15735, 15736, 15738, 15762, 15774, 15775, 15776, 15777, 15778, 15780, 15782, 15783, 15785, 15787, 15798, 15802, 15810, 15811, 15812, 15813, 15815, 15817, 15818, 15820, 15821, 15822, 15823, 15824, 15828, 15829, 15831, 15834, 15840, 15843, 15845, 15847, 15848, 15849, 15853, 15854, 15858, 15861, 15864, 15868, 15870, 15872, 15873, 15875, 15879, 15884, 15885, 15887, 15890, 15891, 15896, 15911, 15915, 15916, 15919, 15924, 15925, 15928, 15929, 15931, 15932, 15933, 15934, 15941, 15943, 15944, 15947, 15953, 15957, 15959, 15960, 15961, 15962, 15975, 15976, 15977, 15979, 15980, 15981, 15984, 15986, 15992, 15993, 15998, 16008, 16011, 16012, 16013, 16015, 16016, 16017, 16018, 16019, 16021, 16022, 16026, 16028, 16029, 16030, 16031, 16032, 16033, 16034, 16039, 16040, 16042, 16043, 16045, 16046, 16048, 16056, 16057, 16059, 16060, 16064, 16086, 16110, 16114, 16117, 16120, 16121, 16124, 16127, 16130, 16133, 16134, 16136, 16137, 16139, 16140, 16141, 16142, 16144, 16145, 16148, 16149, 16150, 16156, 16157, 16166, 16167, 16168, 16169, 16171, 16172, 16173, 16175, 16184, 16185, 16189, 16198, 16200, 16201, 16206, 16213, 16215, 16216, 16218, 16219, 16220, 16221, 16227, 16228, 16229, 16230, 16233, 16234, 16236, 16237, 16238, 16239, 16240, 16246, 16256, 16268, 16271, 16274, 16275, 16276, 16277, 16280, 16281, 16285, 16286, 16290, 16291, 16293, 16296, 16298, 16300, 16301, 16304, 16313, 16315, 16317, 16318, 16324, 16330, 16338, 16341, 16343, 16344, 16349, 16358, 16382, 16383, 16384, 16385, 16386, 16387, 16389, 16393, 16394, 16395, 16399, 16405, 16415, 16431, 16438, 16443, 16449, 16456, 16463, 16471, 16489, 16490, 16495, 16497, 16498, 16503, 16506, 16507, 16508, 16520, 16529, 16530, 16531, 16533, 16535, 16536, 16541, 16544, 16548, 16549, 16550, 16551, 16552, 16554, 16555, 16556, 16557, 16558, 16559, 16568, 16570, 16571, 16589, 16602, 16603, 16604, 16605, 16609, 16612, 16616, 16618, 16620, 16622, 16623, 16624, 16625, 16627, 16628, 16629, 16630, 16631, 16632, 16634, 16635, 16636, 16638, 16639, 16640, 16641, 16642, 16643, 16644, 16645, 16648, 16650, 16651, 16652, 16653, 16665, 16666, 16667, 16670, 16673, 16680, 16682, 16687, 16688, 16689, 16691, 16692, 16693, 16695, 16702, 16703, 16704, 16705, 16706, 16708, 16709, 16710, 16715, 16716, 16717, 16720, 16722, 16730, 16733, 16734, 16735, 16736, 16741, 16742, 16743, 16744, 16745, 16746, 16747, 16753, 16755, 16756, 16758, 16759, 16763, 16765, 16767, 16768, 16769, 16770, 16771, 16777, 16785, 16789, 16811, 16813, 16814, 16816, 16817, 16821, 16825, 16827, 16828, 16829, 16830, 16831, 16832, 16835, 16836, 16841, 16847, 16863, 16864, 16870, 16873, 16875, 16882, 16888, 16891, 16893, 16895, 16896, 16898, 16899, 16900, 16901, 16903, 16904, 16906, 16907, 16909, 16914, 16922, 16927, 16930, 16934, 16935, 16939, 16944, 16948, 16949, 16952, 16958, 16959, 16962, 16964, 16967, 16975, 16976, 16981, 16983, 16988, 16998, 16999, 17000, 17003, 17006, 17009, 17014, 17015, 17016, 17019, 17023, 17028, 17030, 17034, 17035, 17037, 17039, 17040, 17046, 17057, 17058, 17062, 17067, 17074, 17075, 17084, 17093, 17094, 17096, 17098, 17099, 17106, 17112, 17114, 17115, 17116, 17123, 17124, 17127, 17128, 17132, 17136, 17140, 17144, 17150, 17157, 17168, 17170, 17173, 17176, 17180, 17182, 17186, 17187, 17189, 17190, 17191, 17194, 17195, 17196, 17199, 17202, 17205, 17207, 17211, 17216, 17217, 17218, 17219, 17220, 17224, 17225, 17226, 17229, 17231, 17232, 17236, 17237, 17240, 17241, 17243, 17253, 17261, 17267, 17271, 17275, 17280, 17286, 17287, 17288, 17289, 17290, 17291, 17295, 17299, 17301, 17314, 17321, 17327, 17330, 17332, 17333, 17334, 17335, 17338, 17341, 17346, 17347, 17352, 17365, 17368, 17371, 17387, 17388, 17391, 17392, 17394, 17395, 17396, 17397, 17398, 17399, 17407, 17411, 17416, 17417, 17418, 17422, 17427, 17428, 17430, 17433, 17435, 17436, 17439, 17445, 17447, 17448, 17454, 17456, 17457, 17460, 17462, 17466, 17467, 17469, 17470, 17471, 17473, 17474, 17475, 17478, 17483, 17484, 17485, 17486, 17491, 17493, 17495, 17499, 17501, 17503, 17509, 17511, 17515, 17521, 17523, 17533, 17535, 17536, 17539, 17543, 17544, 17546, 17554, 17556, 17561, 17562, 17571, 17573, 17574, 17585, 17590, 17591, 17597, 17599, 17604, 17605, 17608, 17610, 17613, 17620, 17622, 17630, 17644, 17645, 17656, 17657, 17660, 17668, 17669, 17670, 17673, 17674, 17677, 17678, 17679, 17680, 17683, 17684, 17689, 17692, 17693, 17695, 17696, 17697, 17699, 17704, 17706, 17709, 17710, 17713, 17717, 17720, 17721, 17723, 17727, 17728, 17729, 17730, 17735, 17737, 17738, 17739, 17740, 17741, 17742, 17745, 17746, 17747, 17748, 17749, 17750, 17753, 17754, 17755, 17758, 17759, 17760, 17761, 17763, 17764, 17766, 17767, 17769, 17773, 17775, 17782, 17783, 17784, 17786, 17800, 17806, 17811, 17815, 17821, 17822, 17824, 17826, 17827, 17830, 17842, 17847, 17851, 17853, 17855, 17856, 17858, 17860, 17864, 17865, 17867, 17876, 17877, 17878, 17882, 17885, 17888, 17895, 17901, 17902, 17903, 17904, 17905, 17909, 17910, 17915, 17916, 17917, 17918, 17921, 17922, 17923, 17924, 17925, 17926, 17928, 17932, 17933, 17936, 17937, 17938, 17939, 17942, 17950, 17958, 17964, 17967, 17968, 17970, 17971, 17973, 17974, 17975, 17976, 17977, 17978, 17979, 17980, 17981, 17983, 17984, 17988, 17989, 17990, 17995, 17999, 18005, 18006, 18008, 18009, 18012, 18014, 18015, 18016, 18017, 18018, 18019, 18022, 18023, 18032, 18035, 18039, 18040, 18041, 18042, 18047, 18053, 18055, 18058, 18061, 18070, 18073, 18075, 18076, 18077, 18078, 18079, 18082, 18083, 18084, 18089, 18091, 18095, 18096, 18097, 18099, 18100, 18101, 18102, 18105, 18107, 18108, 18109, 18110, 18113, 18114, 18115, 18116, 18117, 18118, 18119, 18123, 18132, 18142, 18144, 18146, 18148, 18151, 18153, 18155, 18166, 18169, 18179, 18185, 18187, 18193, 18194, 18206, 18208, 18212, 18214, 18217, 18220, 18221, 18222, 18228, 18229, 18231, 18274, 18277, 18309, 18342, 18357, 18359, 18361, 18363, 18364, 18366, 18368, 18371, 18372, 18374, 18379, 18380, 18381, 18382, 18388, 18404, 18409, 18410, 18419, 18421, 18422, 18423, 18429, 18440, 18441, 18456, 18460, 18461, 18463, 18488, 18519, 18524, 18538, 18564, 18565, 18595, 18602, 18603, 18604, 18621, 18624, 18634, 18636, 18642, 18644, 18655, 18656, 18663, 18675, 18686, 18700, 18706, 18724, 18749, 18762, 18766, 18775, 18781, 18782, 18783, 18788, 18790, 18791, 18792, 18793, 18795, 18798, 18804, 18805, 18810, 18812, 18813, 18814, 18815, 18816, 18817, 18823, 18824, 18825, 18829, 18830, 18832, 18834, 18835, 18838, 18839, 18843, 18845, 18846, 18848, 18853, 18854, 18864, 18865, 18872, 18873, 18875, 18876, 18878, 18879, 18880, 18881, 18882, 18885, 18890, 18894, 18899, 18902, 18904, 18905, 18909, 18912, 18914, 18916, 18922, 18925, 18931, 18932, 18933, 18934, 18935, 18949, 18953, 18958, 18966, 18968, 18970, 18972, 18978, 19003, 19016, 19017, 19018, 19019, 19025, 19026, 19028, 19031, 19032, 19041, 19042, 19046, 19049, 19052, 19055, 19059, 19062, 19068, 19071, 19368, 19374, 19375, 19376, 19378, 19380, 19381, 19388, 19394, 19396, 19399, 19401, 19403, 19411, 19412, 19441, 19444, 19448, 19455, 19459, 19460, 19466, 19471, 19474, 19479, 19484, 19485, 19492, 19497, 19506, 19520, 19521, 19524, 19525, 19528, 19530, 19533, 19534, 19538, 19547, 19550, 19551, 19554, 19557, 19558, 19559, 19561, 19562, 19566, 19567, 19569, 19570, 19577, 19591, 19595, 19596, 19597, 19598, 19600, 19601, 19602, 19603, 19604, 19634, 19663, 19691, 19820, 19903, 19932, 19947, 19951, 19956, 19957, 19958, 19959, 19962, 19965, 19969, 19974, 19978, 19981, 19983, 19984, 19988, 19989, 19993, 19996, 19997, 19999, 20003, 20004, 20005, 20020, 20021, 20037, 20038, 20042, 20048, 20050, 20051, 20055, 20056, 20057, 20061, 20067, 20068, 20076, 20078, 20080, 20083, 20087, 20088, 20090, 20094, 20095, 20096, 20097, 20098, 20102, 20112, 20115, 20116, 20118, 20120, 20128, 20132, 20137, 20146, 20147, 20155, 20156, 20158, 20160, 20162, 20163, 20166, 20168, 20170, 20175, 20184, 20185, 20186, 20187, 20188, 20191, 20197, 20199, 20202, 20205, 20207, 20208, 20209, 20210, 20212, 20213, 20214, 20217, 20231, 20245, 20248, 20262, 20265, 20267, 20270, 20271, 20272, 20275, 20283, 20286, 20287, 20290, 20291, 20292, 20307, 20308, 20311, 20314, 20315, 20317, 20319, 20324, 20329, 20330, 20331, 20340, 20341, 20344, 20345, 20348, 20349, 20350, 20354, 20355, 20357, 20360, 20361, 20363, 20366, 20367, 20368, 20371, 20372, 20373, 20380, 20384, 20385, 20388, 20390, 20396, 20397, 20398, 20399, 20401, 20404, 20406, 20410, 20412, 20415, 20416, 20421, 20429, 20430, 20431, 20432, 20434, 20437, 20439, 20440, 20441, 20448, 20449, 20451, 20456, 20457, 20458, 20459, 20461, 20469, 20471, 20476, 20495, 20496, 20500, 20503, 20514, 20515, 20516, 20518, 20519, 20524, 20530, 20531, 20532, 20534, 20535, 20536, 20540, 20543, 20544, 20547, 20553, 20554, 20558, 20561, 20562, 20564, 20565, 20571, 20574, 20579, 20580, 20582, 20585, 20591, 20593, 20594, 20595, 20606, 20607, 20610, 20613, 20614, 20615, 20617, 20621, 20624, 20637, 20638, 20642, 20647, 20649, 20651, 20652, 20653, 20655, 20656, 20658, 20660, 20662, 20665, 20666, 20667, 20669, 20670, 20682, 20683, 20684, 20688, 20689, 20690, 20693, 20695, 20696, 20698, 20702, 20703, 20704, 20712, 20713, 20716, 20721, 20722, 20724, 20732, 20739, 20741, 20743, 20744, 20745, 20748, 20749, 20750, 20755, 20756, 20761, 20763, 20764, 20765, 20767, 20770, 20771, 20772, 20773, 20774, 20776, 20799, 20813, 20815, 20817, 20818, 20819, 20822, 20825, 20832, 20833, 20840, 20841, 20846, 20847, 20849, 20851, 20855, 20856, 20859, 20861, 20862, 20863, 20869, 20871, 20872, 20877, 20883, 20885, 20886, 20891, 20892, 20894, 20895, 20896, 20897, 20898, 20899, 20903, 20908, 20909, 20910, 20911, 20914, 20916, 20919, 20920, 20921, 20925, 20926, 20928, 20929, 20934, 20938, 20939, 20940, 20941, 20944, 20945, 20952, 20954, 20955, 20959, 20963, 20965, 20968, 20970, 20971, 20972, 20973, 20979, 20980, 20983, 20988, 20998, 21001, 21008, 21009, 21020, 21021, 21050, 21072, 21104, 21115, 21120, 21151, 21158, 21160, 21162, 21163, 21182, 21183, 21191, 21193, 21195, 21205, 21212, 21213, 21215, 21216, 21218, 21223, 21224, 21227, 21229, 21233, 21237, 21244, 21245, 21249, 21252, 21255, 21256, 21259, 21261, 21267, 21271, 21276, 21277, 21278, 21282, 21292, 21294, 21295, 21296, 21298, 21299, 21300, 21301, 21312, 21314, 21318, 21321, 21325, 21330, 21333, 21334, 21335, 21341, 21342, 21345, 21351, 21352, 21355, 21357, 21358, 21359, 21362, 21364, 21367, 21370, 21371, 21375, 21379, 21380, 21383, 21387, 21388, 21393, 21394, 21395, 21396, 21397, 21399, 21400, 21401, 21404, 21405, 21406, 21410, 21420, 21422, 21428, 21450, 21452, 21453, 21455, 21459, 21460, 21461, 21462, 21463, 21467, 21473, 21474, 21475, 21476, 21478, 21480, 21481, 21483, 21491, 21493, 21495, 21499, 21503, 21510, 21512, 21513, 21515, 21522, 21523, 21526, 21531, 21532, 21535, 21540, 21541, 21544, 21545, 21547, 21549, 21550, 21551, 21555, 21556, 21557, 21559, 21565, 21567, 21570, 21571, 21579, 21580, 21582, 21588, 21589, 21598, 21601, 21602, 21603, 21606, 21609, 21613, 21617, 21618, 21625, 21628, 21630, 21641, 21675, 21676, 21677, 21678, 21681, 21684, 21686, 21689, 21690, 21691, 21692, 21713, 21722, 21723, 21724, 21725, 21750, 21752, 21755, 21761, 21762, 21764, 21765, 21771, 21775, 21779],
                     'label_2':[28, 62, 65, 91, 120, 164, 171, 187, 257, 267, 278, 280, 302, 313, 318, 319, 323, 512, 621, 653, 822, 869, 872, 873, 889, 895, 900, 936, 937, 974, 979, 1010, 1012, 1031, 1032, 1118, 1145, 1297, 1364, 1366, 1367, 1371, 1391, 1408, 1423, 1424, 1432, 1468, 1525, 1542, 1543, 1549, 1555, 1585, 1586, 1596, 1632, 1651, 1711, 1726, 1738, 1739, 1769, 1905, 1932, 1933, 1967, 1978, 2012, 2035, 2038, 2045, 2051, 2089, 2116, 2158, 2228, 2229, 2231, 2236, 2239, 2251, 2260, 2272, 2294, 2298, 2354, 2360, 2365, 2376, 2414, 2416, 2446, 2456, 2468, 2470, 2502, 2523, 2568, 2571, 2572, 2574, 2575, 2583, 2602, 2608, 2610, 2625, 2656, 2664, 2665, 2666, 2667, 2668, 2683, 2700, 2802, 2839, 2852, 2856, 2857, 2871, 3020, 3024, 3028, 3054, 3057, 3147, 3149, 3194, 3228, 3252, 3254, 3260, 3266, 3350, 3374, 3385, 3389, 3390, 3394, 3395, 3403, 3417, 3424, 3434, 3436, 3485, 3486, 3487, 3492, 3499, 3525, 3529, 3532, 3536, 3548, 3549, 3563, 3581, 3594, 3635, 3637, 3638, 3659, 3661, 3705, 3772, 3786, 3806, 3849, 3851, 3860, 3861, 3863, 3864, 3872, 3897, 3958, 3961, 3983, 3985, 4008, 4032, 4062, 4097, 4098, 4099, 4175, 4205, 4261, 4268, 4277, 4279, 4280, 4283, 4284, 4322, 4331, 4332, 4334, 4337, 4338, 4340, 4341, 4344, 4345, 4349, 4353, 4364, 4386, 4389, 4434, 4435, 4440, 4465, 4466, 4470, 4522, 4621, 4628, 4648, 4733, 4762, 4795, 4848, 4851, 4853, 4854, 4922, 5046, 5083, 5091, 5117, 5127, 5129, 5171, 5196, 5202, 5204, 5238, 5275, 5322, 5357, 5366, 5369, 5370, 5375, 5382, 5440, 5442, 5467, 5483, 5489, 5493, 5596, 5608, 5610, 5618, 5620, 5622, 5673, 5701, 5730, 5732, 5735, 5753, 5767, 5775, 5777, 5785, 5787, 5789, 5793, 5794, 5802, 5884, 6001, 6012, 6013, 6014, 6017, 6058, 6103, 6115, 6173, 6206, 6233, 6235, 6259, 6268, 6280, 6284, 6289, 6291, 6400, 6420, 6421, 6454, 6486, 6487, 6550, 6552, 6556, 6585, 6590, 6627, 6628, 6695, 6751, 6796, 6797, 6818, 6821, 6822, 6823, 6831, 6911, 6915, 6920, 6931, 6935, 6937, 6938, 6945, 6974, 6979, 6980, 6991, 6997, 7002, 7016, 7025, 7029, 7039, 7059, 7065, 7070, 7073, 7086, 7092, 7114, 7122, 7153, 7154, 7159, 7175, 7184, 7188, 7190, 7207, 7210, 7215, 7228, 7229, 7230, 7232, 7267, 7321, 7415, 7422, 7423, 7483, 7527, 7549, 7552, 7559, 7560, 7578, 7587, 7594, 7595, 7598, 7634, 7652, 7676, 7677, 7680, 7681, 7682, 7683, 7684, 7688, 7689, 7706, 7707, 7713, 7757, 7761, 7795, 7803, 7806, 7807, 7808, 7829, 7831, 7874, 7905, 7906, 7908, 7914, 7938, 7939, 7940, 7949, 7950, 7951, 7952, 7965, 7968, 7971, 7979, 7981, 7984, 7994, 8014, 8020, 8021, 8025, 8033, 8035, 8043, 8063, 8064, 8066, 8083, 8110, 8167, 8169, 8170, 8190, 8191, 8193, 8213, 8241, 8244, 8257, 8262, 8284, 8302, 8307, 8333, 8377, 8398, 8418, 8424, 8425, 8426, 8457, 8587, 8588, 8592, 8601, 8605, 8614, 8644, 8658, 8667, 8670, 8679, 8684, 8687, 8688, 8720, 8819, 8860, 8864, 8865, 8880, 8964, 9008, 9009, 9018, 9056, 9061, 9089, 9107, 9111, 9120, 9126, 9127, 9136, 9153, 9165, 9208, 9239, 9240, 9263, 9267, 9269, 9296, 9363, 9383, 9388, 9391, 9430, 9517, 9565, 9568, 9569, 9594, 9633, 9757, 9762, 9784, 9785, 9789, 9807, 9808, 9810, 9814, 9815, 9817, 9819, 9838, 9866, 9888, 9902, 9904, 9915, 9928, 9942, 9957, 9983, 10029, 10031, 10082, 10141, 10146, 10147, 10200, 10213, 10248, 10269, 10292, 10342, 10382, 10387, 10442, 10460, 10511, 10517, 10540, 10542, 10543, 10549, 10580, 10591, 10605, 10620, 10623, 10669, 10747, 10802, 10819, 10829, 10836, 10843, 10845, 10846, 10872, 10938, 10941, 10986, 10993, 11035, 11036, 11038, 11048, 11072, 11080, 11148, 11162, 11163, 11167, 11210, 11212, 11266, 11366, 11402, 11403, 11433, 11449, 11467, 11540, 11553, 11557, 11592, 11593, 11596, 11624, 11627, 11643, 11644, 11649, 11679, 11685, 11688, 11692, 11693, 11697, 11698, 11699, 11700, 11702, 11704, 11744, 11745, 11751, 11767, 11787, 11798, 11816, 11819, 11820, 11823, 11839, 11857, 11869, 11893, 11900, 11922, 11936, 11971, 11973, 11995, 11997, 12015, 12059, 12074, 12077, 12082, 12124, 12168, 12169, 12195, 12221, 12223, 12262, 12268, 12276, 12280, 12281, 12282, 12283, 12285, 12295, 12314, 12324, 12327, 12338, 12351, 12363, 12376, 12384, 12418, 12426, 12451, 12460, 12461, 12463, 12464, 12465, 12466, 12467, 12474, 12475, 12479, 12483, 12500, 12508, 12549, 12557, 12569, 12572, 12573, 12582, 12585, 12600, 12604, 12631, 12633, 12635, 12637, 12638, 12639, 12640, 12644, 12646, 12647, 12648, 12655, 12656, 12667, 12668, 12669, 12671, 12689, 12691, 12694, 12695, 12696, 12701, 12702, 12751, 12753, 12764, 12766, 12770, 12771, 12800, 12801, 12802, 12803, 12804, 12807, 12812, 12820, 12830, 12840, 12846, 12849, 12850, 12851, 12853, 12854, 12855, 12856, 12858, 12859, 12861, 12863, 12864, 12865, 12866, 12867, 12877, 12888, 12917, 12918, 12920, 12924, 12925, 12926, 12928, 12930, 12932, 12933, 12940, 12942, 12943, 12961, 12965, 12967, 12973, 12986, 12987, 12988, 12989, 12994, 13031, 13035, 13046, 13051, 13104, 13108, 13120, 13121, 13122, 13123, 13124, 13149, 13162, 13164, 13172, 13173, 13176, 13177, 13179, 13198, 13199, 13203, 13204, 13221, 13236, 13244, 13258, 13261, 13262, 13269, 13271, 13272, 13273, 13308, 13319, 13351, 13356, 13364, 13365, 13369, 13419, 13427, 13437, 13439, 13451, 13457, 13461, 13472, 13485, 13500, 13503, 13506, 13507, 13516, 13517, 13518, 13519, 13521, 13523, 13526, 13527, 13545, 13549, 13567, 13568, 13592, 13594, 13608, 13609, 13664, 13732, 13822, 13835, 13837, 13850, 13854, 13858, 13864, 13865, 13873, 13874, 13880, 13884, 13885, 13915, 13921, 13926, 13927, 13938, 13941, 13944, 13946, 13956, 13971, 13975, 13976, 13977, 13987, 13988, 13991, 13993, 13995, 13996, 13997, 14002, 14010, 14013, 14014, 14017, 14024, 14025, 14026, 14027, 14028, 14029, 14030, 14040, 14049, 14056, 14057, 14065, 14066, 14068, 14073, 14091, 14129, 14140, 14150, 14188, 14198, 14206, 14209, 14214, 14217, 14246, 14248, 14249, 14250, 14251, 14253, 14258, 14259, 14260, 14278, 14281, 14283, 14297, 14325, 14332, 14340, 14346, 14350, 14367, 14373, 14377, 14398, 14420, 14421, 14422, 14473, 14478, 14479, 14481, 14482, 14485, 14486, 14487, 14490, 14493, 14500, 14502, 14505, 14506, 14514, 14515, 14517, 14523, 14525, 14527, 14531, 14533, 14534, 14536, 14537, 14539, 14541, 14543, 14545, 14551, 14554, 14566, 14569, 14570, 14577, 14579, 14580, 14581, 14592, 14593, 14600, 14606, 14608, 14610, 14617, 14618, 14620, 14628, 14629, 14630, 14631, 14632, 14637, 14651, 14652, 14653, 14656, 14663, 14668, 14669, 14673, 14676, 14677, 14678, 14680, 14683, 14685, 14686, 14687, 14688, 14708, 14709, 14717, 14731, 14734, 14737, 14742, 14751, 14752, 14757, 14770, 14774, 14780, 14786, 14790, 14798, 14810, 14816, 14822, 14835, 14840, 14846, 14855, 14876, 14887, 14910, 14921, 14927, 14932, 14939, 14943, 14946, 14959, 14963, 14964, 14971, 14980, 14981, 14982, 14983, 14992, 15005, 15012, 15102, 15111, 15115, 15185, 15188, 15191, 15199, 15217, 15219, 15221, 15224, 15226, 15231, 15234, 15244, 15248, 15249, 15256, 15257, 15261, 15263, 15264, 15269, 15273, 15276, 15279, 15284, 15287, 15293, 15294, 15297, 15302, 15347, 15354, 15368, 15392, 15393, 15397, 15398, 15399, 15400, 15409, 15411, 15412, 15414, 15418, 15422, 15424, 15427, 15428, 15431, 15432, 15433, 15434, 15436, 15437, 15458, 15460, 15480, 15487, 15490, 15501, 15518, 15536, 15538, 15548, 15560, 15572, 15577, 15600, 15608, 15610, 15617, 15624, 15634, 15638, 15650, 15661, 15665, 15678, 15701, 15705, 15708, 15709, 15710, 15711, 15718, 15720, 15721, 15732, 15761, 15767, 15799, 15804, 15805, 15814, 15846, 15850, 15859, 15867, 15869, 15876, 15878, 15910, 15912, 15921, 15949, 15985, 15995, 16000, 16005, 16006, 16010, 16038, 16052, 16053, 16061, 16070, 16091, 16112, 16113, 16159, 16161, 16162, 16177, 16181, 16182, 16187, 16202, 16203, 16208, 16209, 16212, 16214, 16232, 16247, 16248, 16266, 16299, 16328, 16340, 16350, 16352, 16353, 16359, 16374, 16377, 16388, 16392, 16401, 16402, 16412, 16416, 16417, 16418, 16419, 16422, 16424, 16425, 16426, 16427, 16428, 16429, 16430, 16432, 16433, 16434, 16437, 16439, 16442, 16444, 16445, 16447, 16450, 16451, 16455, 16457, 16458, 16459, 16461, 16464, 16465, 16466, 16467, 16469, 16470, 16474, 16476, 16477, 16478, 16480, 16481, 16487, 16491, 16501, 16522, 16545, 16560, 16561, 16562, 16563, 16564, 16566, 16572, 16573, 16574, 16579, 16580, 16581, 16583, 16587, 16588, 16590, 16591, 16592, 16593, 16600, 16601, 16633, 16637, 16647, 16649, 16690, 16700, 16711, 16718, 16719, 16723, 16724, 16726, 16731, 16737, 16786, 16787, 16788, 16795, 16796, 16807, 16812, 16833, 16842, 16843, 16846, 16850, 16851, 16852, 16858, 16859, 16860, 16862, 16865, 16867, 16871, 16877, 16878, 16879, 16880, 16881, 16884, 16885, 16886, 16887, 16889, 16890, 16908, 16917, 16919, 16920, 16921, 16941, 16947, 16951, 16960, 16963, 16965, 16971, 16972, 16973, 16989, 16995, 17002, 17045, 17048, 17049, 17054, 17061, 17073, 17086, 17087, 17088, 17105, 17108, 17117, 17119, 17135, 17181, 17188, 17206, 17249, 17250, 17252, 17254, 17258, 17262, 17263, 17268, 17293, 17326, 17328, 17345, 17355, 17356, 17359, 17361, 17362, 17363, 17364, 17366, 17373, 17374, 17378, 17382, 17383, 17384, 17385, 17402, 17415, 17426, 17429, 17438, 17455, 17458, 17459, 17464, 17465, 17472, 17504, 17506, 17517, 17520, 17524, 17532, 17537, 17538, 17540, 17541, 17550, 17551, 17552, 17553, 17555, 17558, 17560, 17564, 17565, 17566, 17567, 17568, 17570, 17572, 17586, 17600, 17607, 17611, 17612, 17623, 17624, 17625, 17626, 17627, 17628, 17629, 17632, 17633, 17635, 17637, 17638, 17639, 17640, 17641, 17642, 17643, 17646, 17655, 17685, 17687, 17790, 17791, 17794, 17807, 17816, 17818, 17819, 17844, 17869, 17870, 17871, 17872, 17875, 17889, 17891, 17906, 17930, 17934, 17941, 17957, 17959, 17969, 17985, 17992, 18011, 18050, 18062, 18063, 18064, 18065, 18067, 18071, 18085, 18086, 18093, 18098, 18120, 18131, 18134, 18136, 18137, 18138, 18139, 18143, 18145, 18149, 18152, 18154, 18156, 18161, 18164, 18170, 18172, 18174, 18177, 18178, 18183, 18184, 18188, 18190, 18195, 18196, 18197, 18198, 18199, 18200, 18201, 18202, 18203, 18204, 18205, 18207, 18223, 18275, 18282, 18303, 18352, 18362, 18459, 18493, 18514, 18517, 18569, 18629, 18630, 18631, 18632, 18633, 18641, 18649, 18650, 18651, 18665, 18666, 18667, 18692, 18694, 18696, 18697, 18698, 18699, 18708, 18710, 18712, 18713, 18715, 18717, 18718, 18722, 18727, 18764, 18770, 18799, 18811, 18818, 18862, 18893, 18897, 18898, 18906, 18907, 18910, 18915, 18923, 18952, 18957, 18964, 18967, 18983, 18984, 18986, 18987, 18988, 18989, 18991, 18993, 18994, 18995, 18999, 19000, 19001, 19004, 19005, 19008, 19009, 19010, 19014, 19015, 19023, 19036, 19039, 19047, 19048, 19051, 19061, 19065, 19066, 19069, 19196, 19357, 19361, 19362, 19363, 19364, 19365, 19366, 19367, 19369, 19370, 19373, 19384, 19389, 19395, 19400, 19402, 19404, 19407, 19408, 19410, 19416, 19420, 19430, 19432, 19438, 19447, 19452, 19457, 19463, 19468, 19473, 19478, 19481, 19491, 19493, 19498, 19504, 19513, 19514, 19517, 19539, 19571, 19572, 19573, 19574, 19575, 19576, 19593, 19608, 19692, 19721, 19726, 19734, 19737, 19750, 19765, 19766, 19799, 19814, 19830, 19888, 19895, 19904, 19909, 19937, 19943, 19950, 19960, 19964, 19970, 19971, 19972, 20002, 20012, 20031, 20035, 20036, 20045, 20054, 20064, 20066, 20070, 20072, 20079, 20085, 20086, 20117, 20131, 20134, 20138, 20149, 20177, 20201, 20224, 20254, 20255, 20260, 20306, 20310, 20312, 20313, 20320, 20342, 20351, 20356, 20362, 20365, 20374, 20436, 20460, 20465, 20467, 20468, 20470, 20475, 20477, 20478, 20480, 20489, 20510, 20529, 20533, 20556, 20560, 20563, 20566, 20573, 20584, 20596, 20600, 20602, 20640, 20643, 20686, 20691, 20699, 20707, 20710, 20726, 20730, 20737, 20778, 20779, 20780, 20781, 20782, 20783, 20784, 20785, 20786, 20787, 20788, 20790, 20791, 20792, 20793, 20794, 20795, 20796, 20797, 20798, 20800, 20801, 20802, 20803, 20804, 20805, 20807, 20808, 20809, 20843, 20850, 20867, 20874, 20884, 20922, 20923, 20930, 20958, 20960, 20969, 20978, 20984, 21002, 21055, 21087, 21117, 21152, 21159, 21161, 21178, 21181, 21185, 21186, 21187, 21194, 21200, 21201, 21203, 21210, 21220, 21221, 21222, 21225, 21228, 21242, 21253, 21257, 21258, 21264, 21270, 21285, 21319, 21323, 21324, 21327, 21328, 21337, 21339, 21354, 21356, 21361, 21365, 21366, 21378, 21381, 21403, 21413, 21414, 21415, 21416, 21417, 21418, 21419, 21497, 21498, 21500, 21504, 21505, 21506, 21507, 21508, 21509, 21514, 21519, 21539, 21568, 21569, 21576, 21583, 21594, 21595, 21607, 21624, 21679, 21695, 21696, 21715, 21718, 21721, 21730, 21731, 21768, 21772, 21776],
                     'label_3':[20, 38, 72, 75, 82, 133, 143, 144, 180, 182, 185, 186, 188, 190, 191, 195, 197, 198, 199, 200, 201, 203, 204, 205, 206, 208, 209, 211, 216, 218, 219, 221, 222, 223, 226, 229, 230, 233, 234, 237, 240, 241, 242, 243, 246, 248, 250, 251, 252, 253, 254, 256, 258, 259, 260, 261, 262, 263, 264, 265, 266, 270, 453, 469, 477, 480, 482, 545, 557, 894, 896, 907, 908, 914, 915, 916, 918, 926, 929, 966, 1028, 1299, 1309, 1310, 1345, 1360, 1361, 1374, 1380, 1404, 1522, 1523, 1539, 1540, 1551, 1580, 1587, 1588, 1589, 1606, 1607, 1636, 1639, 1654, 1659, 1672, 1674, 1679, 1685, 1709, 1725, 1730, 1743, 1744, 1794, 1798, 1822, 1836, 1838, 1855, 2022, 2084, 2133, 2175, 2326, 2438, 2454, 2584, 2642, 2734, 2737, 2741, 2748, 2778, 2860, 2949, 2959, 3041, 3042, 3047, 3067, 3138, 3141, 3182, 3184, 3187, 3189, 3190, 3214, 3216, 3231, 3244, 3245, 3258, 3278, 3290, 3349, 3376, 3500, 3586, 3639, 3740, 3761, 3882, 3908, 3993, 4041, 4091, 4380, 4404, 4408, 4501, 4517, 4557, 4558, 4591, 4615, 4638, 4686, 4702, 4771, 4839, 4902, 4910, 4932, 4935, 4944, 5011, 5055, 5075, 5093, 5134, 5146, 5169, 5193, 5194, 5205, 5245, 5251, 5281, 5319, 5443, 5621, 5639, 5640, 5643, 5671, 5838, 5840, 5841, 5857, 5896, 5983, 6011, 6161, 6163, 6164, 6166, 6219, 6264, 6339, 6352, 6565, 6616, 6630, 6865, 6867, 6973, 7272, 7273, 7291, 7476, 7636, 7641, 7797, 7798, 7813, 7825, 7873, 7875, 7926, 7931, 7944, 7945, 7946, 7947, 7962, 7972, 7975, 7993, 7995, 8003, 8022, 8023, 8056, 8141, 8248, 8328, 8446, 8460, 8461, 8469, 8472, 8478, 8481, 8642, 8654, 8665, 8725, 8746, 8763, 8765, 8767, 8776, 8777, 8868, 8890, 9028, 9055, 9102, 9112, 9137, 9155, 9170, 9229, 9230, 9248, 9255, 9261, 9262, 9265, 9273, 9284, 9347, 9467, 9502, 9580, 9600, 9601, 9653, 9671, 9688, 9788, 9816, 9823, 9827, 9828, 9830, 9833, 9837, 9840, 9841, 9842, 9925, 9931, 9971, 9975, 10107, 10155, 10164, 10196, 10216, 10294, 10302, 10372, 10391, 10422, 10455, 10504, 10536, 10555, 10572, 10614, 10625, 10628, 10632, 10633, 10634, 10635, 10639, 10640, 10642, 10643, 10645, 10646, 10647, 10648, 10649, 10650, 10654, 10655, 10657, 10659, 10660, 10661, 10662, 10664, 10666, 10667, 10668, 10671, 10673, 10675, 10678, 10680, 10682, 10683, 10684, 10687, 10688, 10689, 10691, 10692, 10693, 10694, 10695, 10698, 10699, 10700, 10701, 10702, 10703, 10705, 10706, 10707, 10708, 10709, 10710, 10711, 10714, 10716, 10717, 10718, 10719, 10720, 10721, 10722, 10723, 10724, 10729, 10730, 10734, 10736, 10737, 10738, 10739, 10740, 10741, 10742, 10743, 10745, 10746, 10748, 10749, 10752, 10753, 10754, 10755, 10756, 10757, 10758, 10759, 10761, 10762, 10763, 10765, 10767, 10768, 10769, 10770, 10771, 10772, 10773, 10774, 10776, 10777, 10778, 10779, 10782, 10783, 10784, 10786, 10787, 10788, 10789, 10790, 10791, 10792, 10793, 10794, 10800, 10824, 10835, 10912, 10994, 11028, 11073, 11206, 11208, 11220, 11225, 11233, 11234, 11267, 11273, 11374, 11393, 11463, 11572, 11578, 11580, 11581, 11582, 11583, 11584, 11585, 11586, 11587, 11588, 11598, 11601, 11603, 11611, 11615, 11619, 11625, 11634, 11650, 11686, 11690, 11691, 11711, 11765, 11768, 11782, 11792, 11831, 11914, 11935, 11940, 11960, 11963, 11968, 11969, 11976, 11977, 11986, 11990, 12003, 12017, 12034, 12035, 12053, 12065, 12069, 12075, 12076, 12094, 12103, 12104, 12107, 12111, 12119, 12136, 12139, 12146, 12155, 12210, 12232, 12238, 12250, 12271, 12287, 12293, 12296, 12298, 12299, 12300, 12313, 12319, 12340, 12341, 12355, 12361, 12375, 12380, 12381, 12385, 12387, 12391, 12393, 12401, 12404, 12406, 12420, 12423, 12431, 12432, 12447, 12491, 12492, 12495, 12504, 12514, 12516, 12518, 12522, 12530, 12531, 12598, 12599, 12602, 12603, 12662, 12714, 12736, 12744, 12747, 12754, 12767, 12780, 12874, 12891, 12911, 13012, 13032, 13038, 13141, 13174, 13183, 13228, 13280, 13283, 13289, 13298, 13302, 13306, 13329, 13388, 13390, 13391, 13392, 13393, 13396, 13402, 13414, 13415, 13418, 13428, 13429, 13430, 13447, 13448, 13475, 13514, 13515, 13555, 13557, 13631, 13637, 13671, 13674, 13701, 13702, 13704, 13705, 13706, 13709, 13725, 13726, 13735, 13736, 13743, 13745, 13747, 13752, 13753, 13758, 13770, 13784, 13795, 13813, 13849, 13856, 13886, 13887, 13888, 13891, 13903, 13922, 13966, 14050, 14077, 14165, 14166, 14173, 14222, 14268, 14290, 14291, 14309, 14317, 14319, 14327, 14328, 14343, 14344, 14376, 14401, 14402, 14403, 14416, 14522, 14672, 14675, 14679, 14703, 14714, 14716, 14729, 14730, 14735, 14740, 14747, 14748, 14763, 14765, 14767, 14772, 14777, 14778, 14788, 14791, 14792, 14795, 14796, 14797, 14800, 14850, 14895, 14915, 14925, 14930, 15075, 15093, 15114, 15166, 15174, 15176, 15237, 15238, 15253, 15262, 15283, 15285, 15299, 15303, 15304, 15375, 15376, 15495, 15498, 15499, 15509, 15515, 15571, 15573, 15582, 15587, 15590, 15591, 15597, 15604, 15620, 15623, 15683, 15714, 15715, 15733, 15734, 15772, 15789, 15801, 15863, 15889, 15923, 15930, 15965, 15967, 15968, 15987, 15990, 16003, 16004, 16020, 16094, 16131, 16244, 16250, 16252, 16289, 16292, 16294, 16309, 16316, 16351, 16354, 16365, 16369, 16376, 16378, 16379, 16397, 16436, 16514, 16526, 16594, 16668, 16707, 16713, 16748, 16749, 16750, 16762, 16794, 16803, 16805, 16818, 16915, 16916, 16929, 16937, 16970, 16974, 16978, 17005, 17029, 17042, 17051, 17052, 17082, 17089, 17103, 17109, 17111, 17113, 17126, 17138, 17139, 17141, 17142, 17147, 17151, 17155, 17160, 17178, 17183, 17203, 17246, 17270, 17317, 17318, 17319, 17322, 17323, 17324, 17325, 17329, 17340, 17342, 17343, 17348, 17376, 17393, 17414, 17453, 17481, 17492, 17526, 17542, 17578, 17580, 17593, 17658, 17688, 17732, 17733, 17771, 17792, 17825, 17828, 17831, 17833, 17843, 17846, 17862, 17896, 17900, 18010, 18031, 18126, 18189, 18224, 18225, 18233, 18244, 18255, 18314, 18373, 18375, 18511, 18537, 18608, 18616, 18639, 18676, 18680, 18729, 18733, 18734, 18735, 18736, 18737, 18738, 18740, 18741, 18742, 18751, 18752, 18753, 18755, 18758, 18759, 18763, 18767, 18768, 18769, 18771, 18772, 18773, 18908, 18926, 18927, 18950, 18959, 18960, 18961, 18971, 19024, 19027, 19035, 19040, 19053, 19067, 19079, 19081, 19082, 19086, 19090, 19120, 19122, 19125, 19145, 19155, 19234, 19264, 19318, 19340, 19377, 19515, 19555, 19599, 19618, 19619, 19637, 19698, 19699, 19717, 19735, 19921, 19942, 19994, 19998, 20009, 20023, 20047, 20104, 20114, 20133, 20165, 20195, 20196, 20198, 20250, 20276, 20288, 20316, 20332, 20333, 20334, 20376, 20387, 20442, 20474, 20501, 20506, 20583, 20611, 20619, 20633, 20641, 20675, 20697, 20701, 20718, 20719, 20720, 20723, 20775, 20827, 20985, 20990, 20991, 20992, 20993, 20994, 20996, 20997, 21019, 21051, 21061, 21086, 21140, 21143, 21144, 21165, 21208, 21209, 21211, 21235, 21265, 21268, 21273, 21274, 21284, 21291, 21293, 21302, 21304, 21305, 21306, 21307, 21311, 21343, 21426, 21432, 21440, 21446, 21448, 21468, 21469, 21470, 21496, 21524, 21533, 21538, 21564, 21605, 21619, 21632, 21633, 21639, 21658, 21659, 21660, 21670, 21672, 21698, 21719, 21737, 21777, 21778]}
        
        label = query_string_dict.get('news')[0]
        labellist = label_dic[query_string_dict.get('news')[0]]
        
        result_message_array = findnews(label, labellist)
        
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
    elif 'interview_q1' in query_string_dict:
        result_message_array =[]
        replyJsonPath = "素材/面試捕手問題一/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)

        # 發送
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
        
    elif 'interview_start' in query_string_dict:
        sorry = {
                "type": "text",
                "text": "很抱歉，因無法偵測到指定攝影裝置（樹莓派），此功能暫停使用!"
        }
        result_message_array = TextSendMessage.new_from_json_dict(sorry)
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
        #pass
    
    elif 'interview_stop' in query_string_dict:
        sorry = {
                "type": "text",
                "text": "很抱歉，因無法偵測到指定攝影裝置（樹莓派），此功能暫停使用!"
        }
        result_message_array = TextSendMessage.new_from_json_dict(sorry)
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
        #result_message_array =[]
        #replyJsonPath = "素材/面試捕手結束面試/reply.json"
        #result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
        
        #line_bot_api.reply_message(
        #    event.reply_token,
        #    result_message_array
        #)

        #pass
    
    elif 'interview_feedback' in query_string_dict:
        
        sorry = {
                "type": "text",
                "text":"很抱歉，因無法偵測到指定攝影裝置（樹莓派），此功能暫停使用!"
        }
        
        result_message_array = TextSendMessage.new_from_json_dict(sorry)
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
        '''
        result_message_array = []
        
        picture = [{
              "type": "text",
              "text": "根據您剛的面試，在整趟的過程中，判斷如下：\n正面｜\n　頭部\n　78%正常/12%左偏/10%右偏\n　身體\n　50%正常/40%左傾/10%右傾\n　肩膀\n　70%正常/30%傾斜\n側身｜\n　駝背\n　70%正常/30%駝背\n　前後傾斜\n　50%正常/0%前傾/50%後傾\n表情｜\n　情緒\n　70%正常（如微笑）/30%緊張"
            },
            {
              "type": "text",
              "text": "下張圖示為正面／身體，有狀況的範例圖"
            },
            {
              "type": "image",
              "originalContentUrl": server_url+"/%E7%B4%A0%E6%9D%90/pictures/1_front.png",
              "previewImageUrl": server_url+"/%E7%B4%A0%E6%9D%90/pictures/1_front.png"
            },
            {
              "type": "text",
              "text": "下張圖示為側面／前後傾斜，有狀況的範例圖"
            },
            {
              "type": "image",
              "originalContentUrl": server_url+"/%E7%B4%A0%E6%9D%90/pictures/1_side.png",
              "previewImageUrl": server_url+"/%E7%B4%A0%E6%9D%90/pictures/1_side.png"
            
        }]
        
        result_message_array = detect_json_array_to_new_message_array_2(picture)
        
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
        '''
    elif 'menu' in query_string_dict:
 
        linkRichMenuId = open("素材/"+query_string_dict.get('menu')[0]+'/rich_menu_id', 'r').read()
        line_bot_api.link_rich_menu_to_user(event.source.user_id,linkRichMenuId)
        
        replyJsonPath = '素材/'+query_string_dict.get('menu')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
  
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )

In [9]:
'''

handler處理image event
先回覆用戶文字消息，並從line上將照片拿回
進入cnn模型運算，判斷照片類型

'''
with open('./models/cnn.pickle', 'rb') as f:
    cnn = pickle.load(f)
with open('./models/cnn1000.pickle', 'rb') as f:
    cnn1000 = pickle.load(f)
    
#with open('./models/cnn-2.pickle', 'rb') as f:
#        cnn2 = pickle.load(f)

@handler.add(MessageEvent, message=ImageMessage)
def handle_message(event):
    returnArray = []
    message_content = line_bot_api.get_message_content(event.message.id)
    
    with open('./images/' + event.message.id + '.jpg', 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)
        
    #輸入圖片目錄路徑
    TEST = glob.glob('./images/' + event.message.id + '.jpg')
    #圖片預處理
    image = load_img(TEST[0], target_size=(224, 224)).convert("RGB")
    img_np2 = np.array(image)
    img_pre2 = preprocess_input(img_np2)
    
    pre2imgs = []
    pre2imgs.append(img_pre2)
    pre2imgs = np.array(pre2imgs)
    pre2imgs
    
    #CNN Model
    pre = cnn.predict(pre2imgs)[0]
    pre = pre.tolist()
    if max(pre) < 0.7:
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text='無法清楚辨別此張照片，請換另一張人像照，謝謝')
        )
    else:
        # label = ['0證件照','1學士照','2生活照']
        no = pre.index(max(pre))  #返回最大值位置
        if no == 0:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='這張照片，給人相對正式的感覺，適合使用')
            )
        elif no == 1:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='這張照片，雖然正式，但偏向學士照，容易給人菜逼八的形象，如果有更好的證件照，建議改選')
            )
        elif no == 2:
            feedback1 = {
                "type": "text",
                "text": "這張照片偏向生活照，如果距離適中，人臉輪廓清晰的話，或許可選擇，但仍建議您使用正式的照片。此外，本照片經過我進一步分析後，結果如下"
            }
            returnArray.insert(0, TextSendMessage.new_from_json_dict(feedback1))
            
            image_data = img_to_array(image)
            image_data = image_data.reshape((1,) + image_data.shape)
            image_data = preprocess_input(image_data)
            pre2 = cnn1000.predict(image_data)
            results = decode_predictions(pre2, top=3)
            
            feedback2 = {
                "type": "text",
                "text": "這張照片有絕大的比例像是：" + str(results[0][0][1]) + "。\n確定這是您想要的嗎？"
            }
            returnArray.append(TextSendMessage.new_from_json_dict(feedback2))
            
            line_bot_api.reply_message(
                event.reply_token,
                returnArray
            )
        #elif no == 3:
        #    line_bot_api.reply_message(
        #        event.reply_token,
        #        TextSendMessage(text='這張照片...似乎不是人，請您再確認一下')
        #    ) 
            

/opt/conda/lib/python3.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [ ]:
'''

Application 運行（開發版）

'''
if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.29.0.2 - - [25/Nov/2019 05:44:27] "POST / HTTP/1.1" 200 -
Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.059 seconds.
Prefix dict has been built succesfully.
172.29.0.2 - - [25/Nov/2019 05:44:34] "POST / HTTP/1.1" 200 -
172.29.0.2 - - [25/Nov/2019 05:44:39] "POST / HTTP/1.1" 200 -
172.29.0.2 - - [25/Nov/2019 05:44:49] "POST / HTTP/1.1" 200 -
172.29.0.2 - - [25/Nov/2019 05:44:55] "POST / HTTP/1.1" 200 -
172.29.0.2 - - [25/Nov/2019 05:45:05] "POST / HTTP/1.1" 200 -
172.29.0.2 - - [25/Nov/2019 05:45:15] "POST / HTTP/1.1" 200 -
172.29.0.2 - - [25/Nov/2019 05:45:16] "POST / HTTP/1.1" 200 -
172.29.0.2 - - [25/Nov/2019 05:45:34] "POST / HTTP/1.1" 200 -
172.29.0.2 - - [25/Nov/2019 05:45:38] "POST / HTTP/1.1" 200 -


40960/35363 [==================================] - 0s 0us/step


172.29.0.2 - - [25/Nov/2019 05:45:46] "POST / HTTP/1.1" 200 -
172.29.0.2 - - [25/Nov/2019 05:45:51] "POST / HTTP/1.1" 200 -


In [ ]:
'''

Application 運行（heroku版）

'''

# import os
# if __name__ == "__main__":
#     app.run(host='0.0.0.0',port=os.environ['PORT'])

In [ ]:
# import requests

# # data = {'username':'markchang', 'passwd':'12345'}
# data = {
#     'loginForm_username': 'pckao',
#     'loginForm_password': 'db103iii'
# }
# response = requests.post('https://ieknet.iek.org.tw/member/loginAction.aspx', data)
# import sys
# import webbrowser
# url = 'https://ieknet.iek.org.tw/ieknews/news_detail.aspx?actiontype=ieknews&indu_idno=12&nsl_id=26f086bbb4184db5aa81e8a06c393cbc'
# webbrowser.open_new_tab(url)